In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import statistics
import math
from scipy.special import logsumexp

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

In [4]:
# All Channels   (CONtv, ComedyDynamics, Docurama, Dove, ElRey, BloodyDisgusting, SoReal,Towie, LoneStar, FilmDetective)

In [5]:
Channel= 'SoReal'
EOM= '2022-08-31'

# ElREy and Towie '_ET'
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove' or Channel == 'BloodyDisgusting' or Channel == 'SoReal'):
    ET=''
    Timezone= 'West'
else:
    ET='_ET'
    Timezone= 'East'
    
filepathM='C:/Users/sortiz/OneDrive - Cinedigm/Databases/Linear/Channel Masters glossary/' +Channel+' Masters/'
filepathA='C:/Users/sortiz/OneDrive - Cinedigm/Databases/Linear/Amagi_hourly_noContent'+ET+'/'
    
glossary_channel =pd.read_excel('C:/Users/sortiz/OneDrive - Cinedigm/Databases/Glossary.xlsx',sheet_name='Platform_Channel')

In [6]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):
    #Dates
    Date_span= 'Jul21_Aug22'
    M_list = ['July_2021','August_2021','September_2021','October_2021','November_2021','December_2021',
              'January_2022','February_2022','March_2022','April_2022','May_2022','June_2022','July_2022','August_2022']
    
elif (Channel == 'ElRey' or Channel == 'SoReal'):
    #Dates
    Date_span= 'Oct21_Aug22'    
    M_list = ['October_2021','November_2021','December_2021',
              'January_2022','February_2022','March_2022','April_2022','May_2022','June_2022','July_2022','August_2022']
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    #Dates
    Date_span= 'Jan22_Aug22'
    M_list = ['January_2022','February_2022','March_2022','April_2022','May_2022','June_2022','July_2022','August_2022']
    
else:
    #LoneStar and FilmDetective
    #Dates
    Date_span= 'Aug22'
    M_list = ['August_2022']

In [7]:
M_L=[]
for i in M_list:
    M_L = M_L + [filepathM + 'Master '+ Channel + ' - ' + i + '.xlsx']
    
Amg_L=[]
for i in M_list:
    Amg_L = Amg_L + [filepathA + i + '_hourly_noContent' + ET + '.csv']

In [8]:
Master = pd.concat([pd.read_excel(file) for file in M_L])
glossary_content = pd.concat([pd.read_excel(file, sheet_name= 'Glossary') for file in M_L])
Amagi = pd.concat([pd.read_csv(file) for file in Amg_L])

In [9]:
Amagi['Date']= pd.to_datetime(Amagi['Date'])
#format = '%m/%d/%Y %H:%M'

In [10]:
Amagi

,Date,Channel Name,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer
0,2021-10-01 00:00:00,bloodydisgusting_littlstar,2,120.00,2,NaN,NaN,NaN
1,2021-10-01 00:00:00,BloodyDisgusting_OandO,5,28.00,5,NaN,NaN,NaN
2,2021-10-01 00:00:00,Bloody Disgusting Roku US,1215,7292.00,995,NaN,NaN,NaN
3,2021-10-01 00:00:00,Bloody Disgusting Vizio US,168,1127.00,139,NaN,NaN,NaN
4,2021-10-01 00:00:00,Bobross Plex,43,1255.00,38,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
34472,2022-08-31 23:00:00,SoReal Plex,13,212.85,11,NaN,NaN,NaN
34473,2022-08-31 23:00:00,So Real Stremium US,4,60.20,2,NaN,NaN,NaN
34474,2022-08-31 23:00:00,Soreal TCL,1,0.00,1,NaN,NaN,NaN
34475,2022-08-31 23:00:00,TheCountryNtwrk LG US,604,2006.80,342,NaN,NaN,NaN


In [11]:
Master

,Unnamed: 0,HOUR,SEGMENT ID,ASSET ID,SEASON ID,SEASON TITLE,ASSET ID.1,TYPE,TITLE,YEAR,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1,2021-10-01 00:00:00:00,NaN,CEG10029784,CEG10029780,World of Weird Clean - Season 1,WorldofWeirdClean,episode,Episode 4,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2021-10-01 00:56:51:12,NaN,CEG10029785,CEG10029780,World of Weird Clean - Season 1,WorldofWeirdClean,episode,Episode 5,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2021-10-01 01:54:00:07,NaN,CEG10029786,CEG10029780,World of Weird Clean- Season 1,WorldofWeirdClean,episode,Episode 6,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2021-10-01 02:50:56:24,NaN,CEG10029787,CEG10029787,World of Weird Clean- Season 1,WorldofWeirdClean,episode,Episode 7,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2021-10-01 03:46:39:05,NaN,CEG10028439,CEG10028438,17 Kids And Counting - Season 1,17KidsAndCounting,episode,Episode One,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,825,2022-08-31 19:10:47:24,NaN,1000000002848,NaN,NaN,1000000002848,episode,Joy,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
825,826,2022-08-31 20:07:23:16,NaN,1000000005027,NaN,NaN,1000000005027,episode,Episode 5,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826,827,2022-08-31 21:04:23:03,NaN,1000000005028,NaN,NaN,1000000005028,episode,Episode 6,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
827,828,2022-08-31 21:59:18:26,NaN,1000000005029,NaN,NaN,1000000005029,episode,Episode 7,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
Master= Master.loc[:,('HOUR','ASSET ID')]
Master.rename(columns={'ASSET ID':'Content ID'},inplace=True)

glossary_content = glossary_content.loc[:,('ID', 'TITLE', 'Series/Film')]
glossary_content.rename(columns= {'ID': 'Content ID'}, inplace=True)

In [13]:
glossary_content['TITLE'] = glossary_content.groupby(['Content ID'])['TITLE'].transform('last')

In [14]:
glossary_content = glossary_content.drop_duplicates()

In [15]:
Master = pd.merge(Master, glossary_content, on = 'Content ID', how='left')
Master.rename(columns= {'TITLE':'Title'}, inplace=True)

In [16]:
Master['TS']=pd.to_datetime(Master['HOUR'],format = '%Y-%m-%d %H:%M:%S:%f')

In [17]:
Master= Master.loc[:,('TS', 'Content ID', 'Title', 'Series/Film')]
Master['Hour_Start']= Master['TS'].dt.hour
Master['Minutes_Start']=Master['TS'].dt.minute
Master['Length']= Master['TS'].shift(-1) - Master['TS']
Master['TS End']= Master['TS'].shift(-1)
Master['Hour_End']= Master['TS End'].dt.hour
Master['Minutes_End']= Master['TS End'].dt.minute


In [18]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End
0,2021-10-01 00:00:00.000,CEG10029784,World of Weird,Series,0,0,0 days 00:56:51.120000,2021-10-01 00:56:51.120,0.0,56.0
1,2021-10-01 00:56:51.120,CEG10029785,World of Weird,Series,0,56,0 days 00:57:08.950000,2021-10-01 01:54:00.070,1.0,54.0
2,2021-10-01 01:54:00.070,CEG10029786,World of Weird,Series,1,54,0 days 00:56:56.170000,2021-10-01 02:50:56.240,2.0,50.0
3,2021-10-01 02:50:56.240,CEG10029787,World of Weird,Series,2,50,0 days 00:55:42.810000,2021-10-01 03:46:39.050,3.0,46.0
4,2021-10-01 03:46:39.050,CEG10028439,17 Kids and Counting,Series,3,46,0 days 00:58:08.230000,2021-10-01 04:44:47.280,4.0,44.0
...,...,...,...,...,...,...,...,...,...,...
9437,2022-08-31 19:10:47.240,1000000002848,Hoarders Get Your House In Order,Series,19,10,0 days 00:56:35.920000,2022-08-31 20:07:23.160,20.0,7.0
9438,2022-08-31 20:07:23.160,1000000005027,Tattoo Fixers on Holiday - Clean,Series,20,7,0 days 00:56:59.870000,2022-08-31 21:04:23.030,21.0,4.0
9439,2022-08-31 21:04:23.030,1000000005028,Tattoo Fixers on Holiday - Clean,Series,21,4,0 days 00:54:55.230000,2022-08-31 21:59:18.260,21.0,59.0
9440,2022-08-31 21:59:18.260,1000000005029,Tattoo Fixers on Holiday - Clean,Series,21,59,0 days 00:56:32.980000,2022-08-31 22:55:51.240,22.0,55.0


# Fixing last row

In [19]:
Master.iloc[-1, 7]= EOM + ' 23:59:59.27'
Master['TS End']= pd.to_datetime(Master['TS End'], format = '%Y-%m-%d %H:%M:%S')
Master.iloc[-1,6]= Master.iloc[-1,7]- Master.iloc[-1,0]
Master.iloc[-1,8]= 23
Master.iloc[-1,9]= 59
Master['Hour_End']= Master['Hour_End'].astype(int)

In [20]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End
0,2021-10-01 00:00:00.000,CEG10029784,World of Weird,Series,0,0,0 days 00:56:51.120000,2021-10-01 00:56:51.120,0,56.0
1,2021-10-01 00:56:51.120,CEG10029785,World of Weird,Series,0,56,0 days 00:57:08.950000,2021-10-01 01:54:00.070,1,54.0
2,2021-10-01 01:54:00.070,CEG10029786,World of Weird,Series,1,54,0 days 00:56:56.170000,2021-10-01 02:50:56.240,2,50.0
3,2021-10-01 02:50:56.240,CEG10029787,World of Weird,Series,2,50,0 days 00:55:42.810000,2021-10-01 03:46:39.050,3,46.0
4,2021-10-01 03:46:39.050,CEG10028439,17 Kids and Counting,Series,3,46,0 days 00:58:08.230000,2021-10-01 04:44:47.280,4,44.0
...,...,...,...,...,...,...,...,...,...,...
9437,2022-08-31 19:10:47.240,1000000002848,Hoarders Get Your House In Order,Series,19,10,0 days 00:56:35.920000,2022-08-31 20:07:23.160,20,7.0
9438,2022-08-31 20:07:23.160,1000000005027,Tattoo Fixers on Holiday - Clean,Series,20,7,0 days 00:56:59.870000,2022-08-31 21:04:23.030,21,4.0
9439,2022-08-31 21:04:23.030,1000000005028,Tattoo Fixers on Holiday - Clean,Series,21,4,0 days 00:54:55.230000,2022-08-31 21:59:18.260,21,59.0
9440,2022-08-31 21:59:18.260,1000000005029,Tattoo Fixers on Holiday - Clean,Series,21,59,0 days 00:56:32.980000,2022-08-31 22:55:51.240,22,55.0


# Adding Middle Hours

In [21]:
#When Lenght crosses 3 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+2 , 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==0), 'Middle_1_Hour'] = 23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==1), 'Middle_1_Hour'] = 0


#When Legth crosses 4 hours
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+3, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==0), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==1), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==2), 'Middle_1_Hour']=0


#when Length crosses 5 hours (ex. 3hr15min film starts at 6:50 ends at 10:05, thats 6,7,8,9,10)
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_3_Hour'] = Master['Hour_Start']+3
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_2_Hour'] = Master['Hour_Start']+2
Master.loc[Master['Hour_End']== Master['Hour_Start']+4, 'Middle_1_Hour'] = Master['Hour_Start']+1
#exeptions
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_3_Hour']=23
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_2_Hour']=22
Master.loc[(Master['Hour_Start']== 20) & (Master['Hour_End']==0), 'Middle_1_Hour']=21

Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_3_Hour']=0
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_2_Hour']=23
Master.loc[(Master['Hour_Start']== 21) & (Master['Hour_End']==1), 'Middle_1_Hour']=22

Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_3_Hour']=1
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_2_Hour']=0
Master.loc[(Master['Hour_Start']== 22) & (Master['Hour_End']==2), 'Middle_1_Hour']=23

Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_3_Hour']=2
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_2_Hour']=1
Master.loc[(Master['Hour_Start']== 23) & (Master['Hour_End']==3), 'Middle_1_Hour']=0

In [22]:
#percentages
#First Hour
Master.loc[Master['Hour_Start']== (Master['Hour_End']),   'First'] = Master['Length']/timedelta(minutes=60)
Master.loc[Master['Hour_Start']+1 == Master['Hour_End'] , 'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+2 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+3 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[Master['Hour_Start']+4 == (Master['Hour_End']),'First'] = (60 -Master['TS'].dt.minute)/60


Master.loc[(Master['Hour_Start']== Master['Hour_End']+23),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+22),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+21),'First'] = (60 -Master['TS'].dt.minute)/60
Master.loc[(Master['Hour_Start']== Master['Hour_End']+20),'First'] = (60 -Master['TS'].dt.minute)/60


#Last Hour
Master.loc[Master['Hour_Start'] != Master['Hour_End'] ,   'Last'] = Master['Minutes_End']/60

#Mid Hours if necessarry
Master.loc[pd.notna(Master['Middle_1_Hour']),'Mid_1']=1
Master.loc[pd.notna(Master['Middle_2_Hour']),'Mid_2']=1
Master.loc[pd.notna(Master['Middle_3_Hour']),'Mid_3']=1


In [23]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3
0,2021-10-01 00:00:00.000,CEG10029784,World of Weird,Series,0,0,0 days 00:56:51.120000,2021-10-01 00:56:51.120,0,56.0,NaN,NaN,NaN,0.947533,NaN,NaN,NaN,NaN
1,2021-10-01 00:56:51.120,CEG10029785,World of Weird,Series,0,56,0 days 00:57:08.950000,2021-10-01 01:54:00.070,1,54.0,NaN,NaN,NaN,0.066667,0.900000,NaN,NaN,NaN
2,2021-10-01 01:54:00.070,CEG10029786,World of Weird,Series,1,54,0 days 00:56:56.170000,2021-10-01 02:50:56.240,2,50.0,NaN,NaN,NaN,0.100000,0.833333,NaN,NaN,NaN
3,2021-10-01 02:50:56.240,CEG10029787,World of Weird,Series,2,50,0 days 00:55:42.810000,2021-10-01 03:46:39.050,3,46.0,NaN,NaN,NaN,0.166667,0.766667,NaN,NaN,NaN
4,2021-10-01 03:46:39.050,CEG10028439,17 Kids and Counting,Series,3,46,0 days 00:58:08.230000,2021-10-01 04:44:47.280,4,44.0,NaN,NaN,NaN,0.233333,0.733333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,2022-08-31 19:10:47.240,1000000002848,Hoarders Get Your House In Order,Series,19,10,0 days 00:56:35.920000,2022-08-31 20:07:23.160,20,7.0,NaN,NaN,NaN,0.833333,0.116667,NaN,NaN,NaN
9438,2022-08-31 20:07:23.160,1000000005027,Tattoo Fixers on Holiday - Clean,Series,20,7,0 days 00:56:59.870000,2022-08-31 21:04:23.030,21,4.0,NaN,NaN,NaN,0.883333,0.066667,NaN,NaN,NaN
9439,2022-08-31 21:04:23.030,1000000005028,Tattoo Fixers on Holiday - Clean,Series,21,4,0 days 00:54:55.230000,2022-08-31 21:59:18.260,21,59.0,NaN,NaN,NaN,0.915342,NaN,NaN,NaN,NaN
9440,2022-08-31 21:59:18.260,1000000005029,Tattoo Fixers on Holiday - Clean,Series,21,59,0 days 00:56:32.980000,2022-08-31 22:55:51.240,22,55.0,NaN,NaN,NaN,0.016667,0.916667,NaN,NaN,NaN


# Amagi Master merge Set Up

In [24]:
Amagi.rename(columns={'Channel Name':'Platform/Channel'}, inplace= True)
Amagi= pd.merge(Amagi, glossary_channel, on= 'Platform/Channel', how='left')

In [25]:
#Change Channel Name **************************************************************************************
Amagi=Amagi[Amagi['Channel']== Channel]

In [26]:
Amagi

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel
20,2021-10-01 00:00:00,Cinedigm - So Real - Roku Roku US,1219,6707.00,985,NaN,NaN,NaN,Roku,SoReal
21,2021-10-01 00:00:00,Cinedigm_Soreal_TCL,1,0.00,1,NaN,NaN,NaN,TCL,SoReal
61,2021-10-01 00:00:00,soreal_littlstar,1,60.00,1,NaN,NaN,NaN,Littlstar,SoReal
62,2021-10-01 00:00:00,SoReal Plex,13,164.00,11,NaN,NaN,NaN,Plex,SoReal
82,2021-10-01 01:00:00,Cinedigm - So Real - Roku Roku US,933,5969.00,788,NaN,NaN,NaN,Roku,SoReal
...,...,...,...,...,...,...,...,...,...,...
491156,2022-08-31 22:00:00,Soreal TCL,4,0.82,4,NaN,NaN,NaN,TCL,SoReal
491200,2022-08-31 23:00:00,SoReal_LG,87,335.62,41,NaN,NaN,NaN,LG,SoReal
491201,2022-08-31 23:00:00,SoReal Plex,13,212.85,11,NaN,NaN,NaN,Plex,SoReal
491202,2022-08-31 23:00:00,So Real Stremium US,4,60.20,2,NaN,NaN,NaN,Stremium,SoReal


In [27]:
# REMOVING ROKU**************

# Amagi= Amagi.loc[Amagi['Platform'] != 'Roku',:]

In [28]:
if Channel == 'Towie':
    Amagi.loc[Amagi['Platform'] == 'Roku', ('Total Viewership Minutes','Session Count','Unique Viewers')]== 0
else:
    Amagi= Amagi.loc[Amagi['Platform'] != 'Roku',:]

In [29]:
Amagi['Hour']= Amagi['Date'].dt.hour
Amagi['Day']= Amagi['Date'].dt.day
Amagi['Year']= Amagi['Date'].dt.year
Amagi['Month']= Amagi['Date'].dt.month

<ipython-input-29-d4c9f923da72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Amagi['Hour']= Amagi['Date'].dt.hour
<ipython-input-29-d4c9f923da72>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Amagi['Day']= Amagi['Date'].dt.day
<ipython-input-29-d4c9f923da72>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [30]:
Amagi.loc[Amagi['Day']==1, ('Platform/Channel','Month','Day')].drop_duplicates()

,Platform/Channel,Month,Day
21,Cinedigm_Soreal_TCL,10,1
61,soreal_littlstar,10,1
62,SoReal Plex,10,1
122,So Real Stremium US,10,1
137,Cinedigm_SoReal_KlowdTV,10,1
44884,Cinedigm_SoReal_KlowdTV,11,1
44925,soreal_littlstar,11,1
44926,SoReal Plex,11,1
44927,So Real Stremium US,11,1
45003,Cinedigm_Soreal_TCL,11,1


In [31]:
Platform=Amagi

In [32]:
Amagi_Min=Amagi.groupby(['Year','Month','Day','Hour'])['Total Viewership Minutes'].agg(sum).reset_index()
Amagi_Sess=Amagi.groupby(['Year','Month','Day','Hour'])['Session Count'].agg(sum).reset_index()
Amagi_Min.rename(columns={'Total Viewership Minutes':'Min'}, inplace=True)
Amagi_Sess.rename(columns={'Session Count':'Sess'}, inplace=True)


In [33]:
Master['Day']= Master['TS'].dt.day
Master['Month']= Master['TS'].dt.month
Master['Year']= Master['TS'].dt.year

In [34]:
Master.loc[Master['Month']==2]

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year
3793,2022-02-01 00:00:00.000,CEG10029782,World of Weird,Series,0,0,0 days 00:54:52.240000,2022-02-01 00:54:52.240,0,54.0,NaN,NaN,NaN,0.914511,NaN,NaN,NaN,NaN,1,2,2022
3794,2022-02-01 00:54:52.240,CEG10029783,World of Weird,Series,0,54,0 days 00:54:50.940000,2022-02-01 01:49:43.180,1,49.0,NaN,NaN,NaN,0.100000,0.816667,NaN,NaN,NaN,1,2,2022
3795,2022-02-01 01:49:43.180,CEG10029784,World of Weird,Series,1,49,0 days 00:53:50.070000,2022-02-01 02:43:33.250,2,43.0,NaN,NaN,NaN,0.183333,0.716667,NaN,NaN,NaN,1,2,2022
3796,2022-02-01 02:43:33.250,CEG10029785,World of Weird,Series,2,43,0 days 00:56:08.930000,2022-02-01 03:39:42.180,3,39.0,NaN,NaN,NaN,0.283333,0.650000,NaN,NaN,NaN,1,2,2022
3797,2022-02-01 03:39:42.180,CEG10028413,Air Rescue,Series,3,39,0 days 00:26:36.870000,2022-02-01 04:06:19.050,4,6.0,NaN,NaN,NaN,0.350000,0.100000,NaN,NaN,NaN,1,2,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4572,2022-02-28 19:30:10.000,1000000002833,Hoarders SOS,Series,19,30,0 days 00:54:46.030000,2022-02-28 20:24:56.030,20,24.0,NaN,NaN,NaN,0.500000,0.400000,NaN,NaN,NaN,28,2,2022
4573,2022-02-28 20:24:56.030,1000000002834,Hoarders SOS,Series,20,24,0 days 00:51:28.070000,2022-02-28 21:16:24.100,21,16.0,NaN,NaN,NaN,0.600000,0.266667,NaN,NaN,NaN,28,2,2022
4574,2022-02-28 21:16:24.100,1000000002835,Hoarders SOS,Series,21,16,0 days 00:54:12.950000,2022-02-28 22:10:37.050,22,10.0,NaN,NaN,NaN,0.733333,0.166667,NaN,NaN,NaN,28,2,2022
4575,2022-02-28 22:10:37.050,1000000002836,Hoarders SOS,Series,22,10,0 days 00:54:49.160000,2022-02-28 23:05:26.210,23,5.0,NaN,NaN,NaN,0.833333,0.083333,NaN,NaN,NaN,28,2,2022


# Merging Master with Amagi Data

In [35]:
H=('Hour_Start','Middle_1_Hour','Middle_2_Hour','Middle_3_Hour','Hour_End')
M=('start','Mid1','Mid2','Mid3','End')


for i,j in zip(H,M):
    Master= pd.merge(Master, Amagi_Min ,left_on=['Year','Month','Day',i], right_on=['Year','Month','Day', 'Hour'], how='left' )
    Master= pd.merge(Master, Amagi_Sess ,left_on=['Year','Month','Day',i], right_on=['Year','Month','Day', 'Hour'], how='left' )
    Master.drop(columns=['Hour_x','Hour_y'], inplace= True)
    Master.rename(columns={'Min':'Min_Hour_'+j}, inplace=True)
    Master.rename(columns={'Sess':'Sess_Hour_'+j}, inplace= True)

In [36]:
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Min_Hour_End']= 0
Master.loc[(Master['Hour_Start']== Master['Hour_End']), 'Sess_Hour_End']= 0

In [37]:
Master.loc[Master['Month']==2]

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End
3793,2022-02-01 00:00:00.000,CEG10029782,World of Weird,Series,0,0,0 days 00:54:52.240000,2022-02-01 00:54:52.240,0,54.0,NaN,NaN,NaN,0.914511,NaN,NaN,NaN,NaN,1,2,2022,320.75,31.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
3794,2022-02-01 00:54:52.240,CEG10029783,World of Weird,Series,0,54,0 days 00:54:50.940000,2022-02-01 01:49:43.180,1,49.0,NaN,NaN,NaN,0.100000,0.816667,NaN,NaN,NaN,1,2,2022,320.75,31.0,NaN,NaN,NaN,NaN,NaN,NaN,368.30,36.0
3795,2022-02-01 01:49:43.180,CEG10029784,World of Weird,Series,1,49,0 days 00:53:50.070000,2022-02-01 02:43:33.250,2,43.0,NaN,NaN,NaN,0.183333,0.716667,NaN,NaN,NaN,1,2,2022,368.30,36.0,NaN,NaN,NaN,NaN,NaN,NaN,362.40,25.0
3796,2022-02-01 02:43:33.250,CEG10029785,World of Weird,Series,2,43,0 days 00:56:08.930000,2022-02-01 03:39:42.180,3,39.0,NaN,NaN,NaN,0.283333,0.650000,NaN,NaN,NaN,1,2,2022,362.40,25.0,NaN,NaN,NaN,NaN,NaN,NaN,340.26,37.0
3797,2022-02-01 03:39:42.180,CEG10028413,Air Rescue,Series,3,39,0 days 00:26:36.870000,2022-02-01 04:06:19.050,4,6.0,NaN,NaN,NaN,0.350000,0.100000,NaN,NaN,NaN,1,2,2022,340.26,37.0,NaN,NaN,NaN,NaN,NaN,NaN,316.73,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4572,2022-02-28 19:30:10.000,1000000002833,Hoarders SOS,Series,19,30,0 days 00:54:46.030000,2022-02-28 20:24:56.030,20,24.0,NaN,NaN,NaN,0.500000,0.400000,NaN,NaN,NaN,28,2,2022,1063.90,196.0,NaN,NaN,NaN,NaN,NaN,NaN,842.40,147.0
4573,2022-02-28 20:24:56.030,1000000002834,Hoarders SOS,Series,20,24,0 days 00:51:28.070000,2022-02-28 21:16:24.100,21,16.0,NaN,NaN,NaN,0.600000,0.266667,NaN,NaN,NaN,28,2,2022,842.40,147.0,NaN,NaN,NaN,NaN,NaN,NaN,835.44,110.0
4574,2022-02-28 21:16:24.100,1000000002835,Hoarders SOS,Series,21,16,0 days 00:54:12.950000,2022-02-28 22:10:37.050,22,10.0,NaN,NaN,NaN,0.733333,0.166667,NaN,NaN,NaN,28,2,2022,835.44,110.0,NaN,NaN,NaN,NaN,NaN,NaN,734.36,66.0
4575,2022-02-28 22:10:37.050,1000000002836,Hoarders SOS,Series,22,10,0 days 00:54:49.160000,2022-02-28 23:05:26.210,23,5.0,NaN,NaN,NaN,0.833333,0.083333,NaN,NaN,NaN,28,2,2022,734.36,66.0,NaN,NaN,NaN,NaN,NaN,NaN,713.86,56.0


# Fill nan with 0

In [38]:
L=['Last','Mid_1','Mid_2','Mid_3','Min_Hour_Mid1','Min_Hour_Mid2',
 'Min_Hour_Mid3','Min_Hour_End','Sess_Hour_Mid1','Sess_Hour_Mid2',
 'Sess_Hour_Mid3','Sess_Hour_End']
for i in L:
    Master[i]=Master[i].fillna(0)

# Multiply percent of hour with data for whole hour

In [39]:
Master['Total Minutes']= ((Master['First']*Master['Min_Hour_start']) + 
(Master['Mid_1']*Master['Min_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Min_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Min_Hour_Mid3']) +                           
(Master['Last'] * Master['Min_Hour_End']))

Master['Total Session']= ((Master['First'] * Master['Sess_Hour_start']) + 
(Master['Mid_1'] * Master['Sess_Hour_Mid1']) + 
(Master['Mid_2'] * Master['Sess_Hour_Mid2']) + 
(Master['Mid_3'] * Master['Sess_Hour_Mid3']) + 
(Master['Last'] * Master['Sess_Hour_End']))

Master.drop(Master[Master['TS']==Master['TS End']].index, inplace=True)
Master.reset_index()

,index,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End,Total Minutes,Total Session
0,0,2021-10-01 00:00:00.000,CEG10029784,World of Weird,Series,0,0,0 days 00:56:51.120000,2021-10-01 00:56:51.120,0,56.0,NaN,NaN,NaN,0.947533,0.000000,0.0,0.0,0.0,1,10,2021,224.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,212.247467,14.213000
1,1,2021-10-01 00:56:51.120,CEG10029785,World of Weird,Series,0,56,0 days 00:57:08.950000,2021-10-01 01:54:00.070,1,54.0,NaN,NaN,NaN,0.066667,0.900000,0.0,0.0,0.0,1,10,2021,224.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,195.00,15.0,190.433333,14.500000
2,2,2021-10-01 01:54:00.070,CEG10029786,World of Weird,Series,1,54,0 days 00:56:56.170000,2021-10-01 02:50:56.240,2,50.0,NaN,NaN,NaN,0.100000,0.833333,0.0,0.0,0.0,1,10,2021,195.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,339.00,18.0,302.000000,16.500000
3,3,2021-10-01 02:50:56.240,CEG10029787,World of Weird,Series,2,50,0 days 00:55:42.810000,2021-10-01 03:46:39.050,3,46.0,NaN,NaN,NaN,0.166667,0.766667,0.0,0.0,0.0,1,10,2021,339.00,18.0,0.0,0.0,0.0,0.0,0.0,0.0,376.00,15.0,344.766667,14.500000
4,4,2021-10-01 03:46:39.050,CEG10028439,17 Kids and Counting,Series,3,46,0 days 00:58:08.230000,2021-10-01 04:44:47.280,4,44.0,NaN,NaN,NaN,0.233333,0.733333,0.0,0.0,0.0,1,10,2021,376.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,340.00,10.0,337.066667,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,9437,2022-08-31 19:10:47.240,1000000002848,Hoarders Get Your House In Order,Series,19,10,0 days 00:56:35.920000,2022-08-31 20:07:23.160,20,7.0,NaN,NaN,NaN,0.833333,0.116667,0.0,0.0,0.0,31,8,2022,1073.49,331.0,0.0,0.0,0.0,0.0,0.0,0.0,660.72,276.0,971.659000,308.033333
9438,9438,2022-08-31 20:07:23.160,1000000005027,Tattoo Fixers on Holiday - Clean,Series,20,7,0 days 00:56:59.870000,2022-08-31 21:04:23.030,21,4.0,NaN,NaN,NaN,0.883333,0.066667,0.0,0.0,0.0,31,8,2022,660.72,276.0,0.0,0.0,0.0,0.0,0.0,0.0,943.30,219.0,646.522667,258.400000
9439,9439,2022-08-31 21:04:23.030,1000000005028,Tattoo Fixers on Holiday - Clean,Series,21,4,0 days 00:54:55.230000,2022-08-31 21:59:18.260,21,59.0,NaN,NaN,NaN,0.915342,0.000000,0.0,0.0,0.0,31,8,2022,943.30,219.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,863.441794,200.459825
9440,9440,2022-08-31 21:59:18.260,1000000005029,Tattoo Fixers on Holiday - Clean,Series,21,59,0 days 00:56:32.980000,2022-08-31 22:55:51.240,22,55.0,NaN,NaN,NaN,0.016667,0.916667,0.0,0.0,0.0,31,8,2022,943.30,219.0,0.0,0.0,0.0,0.0,0.0,0.0,554.54,144.0,524.050000,135.650000


# Master Breakdown

In [40]:
Master['First_Hour_Min']=Master['First']*Master['Min_Hour_start']
Master['Second_Hour_Min']=Master['Mid_1']*Master['Min_Hour_Mid1']
Master['Third_Hour_Min']=Master['Mid_2']*Master['Min_Hour_Mid2']
Master['Fourth_Hour_Min']=Master['Mid_3']*Master['Min_Hour_Mid3']
Master['Last_Hour_Min']=Master['Last']*Master['Min_Hour_End']

Master['First_Hour_Sess']=Master['First']*Master['Sess_Hour_start']
Master['Second_Hour_Sess']=Master['Mid_1']*Master['Sess_Hour_Mid1']
Master['Third_Hour_Sess']=Master['Mid_2']*Master['Sess_Hour_Mid2']
Master['Fourth_Hour_Sess']=Master['Mid_3']*Master['Sess_Hour_Mid3']
Master['Last_Hour_Sess']=Master['Last']*Master['Sess_Hour_End']


In [41]:
Master['Length_1']=Master['First']*60
Master['Length_2']=Master['Mid_1']*60
Master['Length_3']=Master['Mid_2']*60
Master['Length_4']=Master['Mid_3']*60
Master['Length_5']=Master['Last']*60

In [42]:
Master

,TS,Content ID,Title,Series/Film,Hour_Start,Minutes_Start,Length,TS End,Hour_End,Minutes_End,Middle_1_Hour,Middle_2_Hour,Middle_3_Hour,First,Last,Mid_1,Mid_2,Mid_3,Day,Month,Year,Min_Hour_start,Sess_Hour_start,Min_Hour_Mid1,Sess_Hour_Mid1,Min_Hour_Mid2,Sess_Hour_Mid2,Min_Hour_Mid3,Sess_Hour_Mid3,Min_Hour_End,Sess_Hour_End,Total Minutes,Total Session,First_Hour_Min,Second_Hour_Min,Third_Hour_Min,Fourth_Hour_Min,Last_Hour_Min,First_Hour_Sess,Second_Hour_Sess,Third_Hour_Sess,Fourth_Hour_Sess,Last_Hour_Sess,Length_1,Length_2,Length_3,Length_4,Length_5
0,2021-10-01 00:00:00.000,CEG10029784,World of Weird,Series,0,0,0 days 00:56:51.120000,2021-10-01 00:56:51.120,0,56.0,NaN,NaN,NaN,0.947533,0.000000,0.0,0.0,0.0,1,10,2021,224.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,212.247467,14.213000,212.247467,0.0,0.0,0.0,0.000000,14.213000,0.0,0.0,0.0,0.000000,56.8520,0.0,0.0,0.0,0.0
1,2021-10-01 00:56:51.120,CEG10029785,World of Weird,Series,0,56,0 days 00:57:08.950000,2021-10-01 01:54:00.070,1,54.0,NaN,NaN,NaN,0.066667,0.900000,0.0,0.0,0.0,1,10,2021,224.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,195.00,15.0,190.433333,14.500000,14.933333,0.0,0.0,0.0,175.500000,1.000000,0.0,0.0,0.0,13.500000,4.0000,0.0,0.0,0.0,54.0
2,2021-10-01 01:54:00.070,CEG10029786,World of Weird,Series,1,54,0 days 00:56:56.170000,2021-10-01 02:50:56.240,2,50.0,NaN,NaN,NaN,0.100000,0.833333,0.0,0.0,0.0,1,10,2021,195.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,339.00,18.0,302.000000,16.500000,19.500000,0.0,0.0,0.0,282.500000,1.500000,0.0,0.0,0.0,15.000000,6.0000,0.0,0.0,0.0,50.0
3,2021-10-01 02:50:56.240,CEG10029787,World of Weird,Series,2,50,0 days 00:55:42.810000,2021-10-01 03:46:39.050,3,46.0,NaN,NaN,NaN,0.166667,0.766667,0.0,0.0,0.0,1,10,2021,339.00,18.0,0.0,0.0,0.0,0.0,0.0,0.0,376.00,15.0,344.766667,14.500000,56.500000,0.0,0.0,0.0,288.266667,3.000000,0.0,0.0,0.0,11.500000,10.0000,0.0,0.0,0.0,46.0
4,2021-10-01 03:46:39.050,CEG10028439,17 Kids and Counting,Series,3,46,0 days 00:58:08.230000,2021-10-01 04:44:47.280,4,44.0,NaN,NaN,NaN,0.233333,0.733333,0.0,0.0,0.0,1,10,2021,376.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,340.00,10.0,337.066667,10.833333,87.733333,0.0,0.0,0.0,249.333333,3.500000,0.0,0.0,0.0,7.333333,14.0000,0.0,0.0,0.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,2022-08-31 19:10:47.240,1000000002848,Hoarders Get Your House In Order,Series,19,10,0 days 00:56:35.920000,2022-08-31 20:07:23.160,20,7.0,NaN,NaN,NaN,0.833333,0.116667,0.0,0.0,0.0,31,8,2022,1073.49,331.0,0.0,0.0,0.0,0.0,0.0,0.0,660.72,276.0,971.659000,308.033333,894.575000,0.0,0.0,0.0,77.084000,275.833333,0.0,0.0,0.0,32.200000,50.0000,0.0,0.0,0.0,7.0
9438,2022-08-31 20:07:23.160,1000000005027,Tattoo Fixers on Holiday - Clean,Series,20,7,0 days 00:56:59.870000,2022-08-31 21:04:23.030,21,4.0,NaN,NaN,NaN,0.883333,0.066667,0.0,0.0,0.0,31,8,2022,660.72,276.0,0.0,0.0,0.0,0.0,0.0,0.0,943.30,219.0,646.522667,258.400000,583.636000,0.0,0.0,0.0,62.886667,243.800000,0.0,0.0,0.0,14.600000,53.0000,0.0,0.0,0.0,4.0
9439,2022-08-31 21:04:23.030,1000000005028,Tattoo Fixers on Holiday - Clean,Series,21,4,0 days 00:54:55.230000,2022-08-31 21:59:18.260,21,59.0,NaN,NaN,NaN,0.915342,0.000000,0.0,0.0,0.0,31,8,2022,943.30,219.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,863.441794,200.459825,863.441794,0.0,0.0,0.0,0.000000,200.459825,0.0,0.0,0.0,0.000000,54.9205,0.0,0.0,0.0,0.0
9440,2022-08-31 21:59:18.260,1000000005029,Tattoo Fixers on Holiday - Clean,Series,21,59,0 days 00:56:32.980000,2022-08-31 22:55:51.240,22,55.0,NaN,NaN,NaN,0.016667,0.916667,0.0,0.0,0.0,31,8,2022,943.30,219.0,0.0,0.0,0.0,0.0,0.0,0.0,554.54,144.0,524.050000,135.650000,15.721667,0.0,0.0,0.0,508.328333,3.650000,0.0,0.0,0.0,132.000000,1.0000,0.0,0.0,0.0,55.0


In [43]:
Master1= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_Start','First_Hour_Min','First_Hour_Sess',
                       'Length_1','Total Minutes','Total Session','Year','Month')]
Master2= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_1_Hour','Second_Hour_Min','Second_Hour_Sess',
                       'Length_2','Total Minutes','Total Session','Year','Month')]
Master3= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_2_Hour','Third_Hour_Min','Third_Hour_Sess',
                       'Length_3','Total Minutes','Total Session','Year','Month')]
Master4= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Middle_3_Hour','Fourth_Hour_Min','Fourth_Hour_Sess',
                       'Length_4','Total Minutes','Total Session','Year','Month')]
Master5= Master.loc[:,('Content ID','TS','TS End','Title','Series/Film','Hour_End','Last_Hour_Min','Last_Hour_Sess',
                       'Length_5','Total Minutes','Total Session','Year','Month')]

In [44]:
Master1.rename(columns={'Hour_Start':'Hour','First_Hour_Min':'Minutes Viewed in Hour','Length_1':'Length(min) in the Hour',
                        'First_Hour_Sess':'Sessions in Hour'},inplace=True)
Master2.rename(columns={'Middle_1_Hour':'Hour','Second_Hour_Min':'Minutes Viewed in Hour','Length_2':'Length(min) in the Hour',
                        'Second_Hour_Sess':'Sessions in Hour'},inplace=True)
Master3.rename(columns={'Middle_2_Hour':'Hour','Third_Hour_Min':'Minutes Viewed in Hour','Length_3':'Length(min) in the Hour',
                        'Third_Hour_Sess':'Sessions in Hour'},inplace=True)
Master4.rename(columns={'Middle_3_Hour':'Hour','Fourth_Hour_Min':'Minutes Viewed in Hour','Length_4':'Length(min) in the Hour',
                        'Fourth_Hour_Sess':'Sessions in Hour'},inplace=True)
Master5.rename(columns={'Hour_End':'Hour','Last_Hour_Min':'Minutes Viewed in Hour','Length_5':'Length(min) in the Hour',
                        'Last_Hour_Sess':'Sessions in Hour'},inplace=True)



In [45]:
DF= Master1.append([Master2,Master3,Master4,Master5])
DF.dropna(inplace=True)
DF['Weekday']= DF['TS'].dt.weekday

In [46]:
DF

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,World of Weird,Series,0.0,212.247467,14.213,56.852,212.247467,14.213000,2021,10,4
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,World of Weird,Series,0.0,14.933333,1.000,4.000,190.433333,14.500000,2021,10,4
2,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,World of Weird,Series,1.0,19.500000,1.500,6.000,302.000000,16.500000,2021,10,4
3,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,World of Weird,Series,2.0,56.500000,3.000,10.000,344.766667,14.500000,2021,10,4
4,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,17 Kids and Counting,Series,3.0,87.733333,3.500,14.000,337.066667,10.833333,2021,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,1000000002848,2022-08-31 19:10:47.240,2022-08-31 20:07:23.160,Hoarders Get Your House In Order,Series,20.0,77.084000,32.200,7.000,971.659000,308.033333,2022,8,2
9438,1000000005027,2022-08-31 20:07:23.160,2022-08-31 21:04:23.030,Tattoo Fixers on Holiday - Clean,Series,21.0,62.886667,14.600,4.000,646.522667,258.400000,2022,8,2
9439,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,Tattoo Fixers on Holiday - Clean,Series,21.0,0.000000,0.000,0.000,863.441794,200.459825,2022,8,2
9440,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,Tattoo Fixers on Holiday - Clean,Series,22.0,508.328333,132.000,55.000,524.050000,135.650000,2022,8,2


In [47]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 3),
        (DF['Weekday'] < 5) & (DF['Hour'] < 13) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 13),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 7) & (DF['Hour'] >= 3),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 17) & (DF['Hour'] >= 7),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 17),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 3) | (DF['Hour'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday'] < 5) & (DF['Hour'] < 9) & (DF['Hour'] >= 6),
        (DF['Weekday'] < 5) & (DF['Hour'] < 16) & (DF['Hour'] >= 9),
        (DF['Weekday'] < 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 16),
        (DF['Weekday'] < 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] < 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] < 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 10) & (DF['Hour'] >= 6),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 20) & (DF['Hour'] >= 10),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 23) & (DF['Hour'] >= 20),
        (DF['Weekday'] >= 5) & ((DF['Hour'] < 2) | (DF['Hour'] >= 23)),
        (DF['Weekday'] >= 5) & (DF['Hour'] < 6) & (DF['Hour'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart']= np.select(Conditions1, Daypart1, default= np.nan)

In [48]:
DF

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday,Daypart
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,World of Weird,Series,0.0,212.247467,14.213,56.852,212.247467,14.213000,2021,10,4,Overnight
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,World of Weird,Series,0.0,14.933333,1.000,4.000,190.433333,14.500000,2021,10,4,Overnight
2,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,World of Weird,Series,1.0,19.500000,1.500,6.000,302.000000,16.500000,2021,10,4,Overnight
3,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,World of Weird,Series,2.0,56.500000,3.000,10.000,344.766667,14.500000,2021,10,4,Overnight
4,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,17 Kids and Counting,Series,3.0,87.733333,3.500,14.000,337.066667,10.833333,2021,10,4,Weekday Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,1000000002848,2022-08-31 19:10:47.240,2022-08-31 20:07:23.160,Hoarders Get Your House In Order,Series,20.0,77.084000,32.200,7.000,971.659000,308.033333,2022,8,2,Late Night
9438,1000000005027,2022-08-31 20:07:23.160,2022-08-31 21:04:23.030,Tattoo Fixers on Holiday - Clean,Series,21.0,62.886667,14.600,4.000,646.522667,258.400000,2022,8,2,Late Night
9439,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,Tattoo Fixers on Holiday - Clean,Series,21.0,0.000000,0.000,0.000,863.441794,200.459825,2022,8,2,Late Night
9440,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,Tattoo Fixers on Holiday - Clean,Series,22.0,508.328333,132.000,55.000,524.050000,135.650000,2022,8,2,Late Night


In [49]:
DF['Minutes per Minute']= DF['Minutes Viewed in Hour']/DF['Length(min) in the Hour']
DF['Sessions per Minute']= DF['Sessions in Hour']/DF['Length(min) in the Hour']
DF['Session Duration']= DF['Minutes Viewed in Hour']/DF['Sessions in Hour']
DF['Day']= DF['TS'].dt.day

In [50]:
DF

,Content ID,TS,TS End,Title,Series/Film,Hour,Minutes Viewed in Hour,Sessions in Hour,Length(min) in the Hour,Total Minutes,Total Session,Year,Month,Weekday,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Day
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,World of Weird,Series,0.0,212.247467,14.213,56.852,212.247467,14.213000,2021,10,4,Overnight,3.733333,0.25,14.933333,1
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,World of Weird,Series,0.0,14.933333,1.000,4.000,190.433333,14.500000,2021,10,4,Overnight,3.733333,0.25,14.933333,1
2,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,World of Weird,Series,1.0,19.500000,1.500,6.000,302.000000,16.500000,2021,10,4,Overnight,3.250000,0.25,13.000000,1
3,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,World of Weird,Series,2.0,56.500000,3.000,10.000,344.766667,14.500000,2021,10,4,Overnight,5.650000,0.30,18.833333,1
4,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,17 Kids and Counting,Series,3.0,87.733333,3.500,14.000,337.066667,10.833333,2021,10,4,Weekday Morning,6.266667,0.25,25.066667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,1000000002848,2022-08-31 19:10:47.240,2022-08-31 20:07:23.160,Hoarders Get Your House In Order,Series,20.0,77.084000,32.200,7.000,971.659000,308.033333,2022,8,2,Late Night,11.012000,4.60,2.393913,31
9438,1000000005027,2022-08-31 20:07:23.160,2022-08-31 21:04:23.030,Tattoo Fixers on Holiday - Clean,Series,21.0,62.886667,14.600,4.000,646.522667,258.400000,2022,8,2,Late Night,15.721667,3.65,4.307306,31
9439,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,Tattoo Fixers on Holiday - Clean,Series,21.0,0.000000,0.000,0.000,863.441794,200.459825,2022,8,2,Late Night,NaN,NaN,NaN,31
9440,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,Tattoo Fixers on Holiday - Clean,Series,22.0,508.328333,132.000,55.000,524.050000,135.650000,2022,8,2,Late Night,9.242333,2.40,3.850972,31


# Month Breakdown

In [51]:
Condwk=[
    DF['Weekday']== 0,DF['Weekday']== 1,DF['Weekday']== 2,DF['Weekday']== 3,
    DF['Weekday']== 4,DF['Weekday']== 5,DF['Weekday']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [52]:
DF= DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday','Weekday(name)','Hour','Series/Film',
             'Title','Daypart','Minutes per Minute','Sessions per Minute','Session Duration','Year','Month')]

In [53]:
DF

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday,Weekday(name),Hour,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Year,Month
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,56.852,1,4,Fri,0.0,Series,World of Weird,Overnight,3.733333,0.25,14.933333,2021,10
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,4.000,1,4,Fri,0.0,Series,World of Weird,Overnight,3.733333,0.25,14.933333,2021,10
2,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,6.000,1,4,Fri,1.0,Series,World of Weird,Overnight,3.250000,0.25,13.000000,2021,10
3,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,10.000,1,4,Fri,2.0,Series,World of Weird,Overnight,5.650000,0.30,18.833333,2021,10
4,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,14.000,1,4,Fri,3.0,Series,17 Kids and Counting,Weekday Morning,6.266667,0.25,25.066667,2021,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,1000000002848,2022-08-31 19:10:47.240,2022-08-31 20:07:23.160,7.000,31,2,Wed,20.0,Series,Hoarders Get Your House In Order,Late Night,11.012000,4.60,2.393913,2022,8
9438,1000000005027,2022-08-31 20:07:23.160,2022-08-31 21:04:23.030,4.000,31,2,Wed,21.0,Series,Tattoo Fixers on Holiday - Clean,Late Night,15.721667,3.65,4.307306,2022,8
9439,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,0.000,31,2,Wed,21.0,Series,Tattoo Fixers on Holiday - Clean,Late Night,NaN,NaN,NaN,2022,8
9440,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,55.000,31,2,Wed,22.0,Series,Tattoo Fixers on Holiday - Clean,Late Night,9.242333,2.40,3.850972,2022,8


In [54]:
DF.dropna(inplace=True)
DF.sort_values(by=['TS','TS End'])
DF['Hour']=DF['Hour'].astype(int)
DF.rename(columns={'Hour':'Hour(24-Pac)','Weekday':'Weekday(num)','Weekday(name)':'Weekday'},inplace=True)

In [55]:
Con4=[
    (DF['Hour(24-Pac)']== 0),(DF['Hour(24-Pac)']== 1),(DF['Hour(24-Pac)']== 2),(DF['Hour(24-Pac)']== 3),
    (DF['Hour(24-Pac)']== 4),(DF['Hour(24-Pac)']== 5),(DF['Hour(24-Pac)']== 6),(DF['Hour(24-Pac)']== 7),
    (DF['Hour(24-Pac)']== 8),(DF['Hour(24-Pac)']== 9),(DF['Hour(24-Pac)']== 10),(DF['Hour(24-Pac)']== 11),
    (DF['Hour(24-Pac)']== 12),(DF['Hour(24-Pac)']== 13),(DF['Hour(24-Pac)']== 14),(DF['Hour(24-Pac)']== 15),
    (DF['Hour(24-Pac)']== 16),(DF['Hour(24-Pac)']== 17),(DF['Hour(24-Pac)']== 18),(DF['Hour(24-Pac)']== 19),
    (DF['Hour(24-Pac)']== 20),(DF['Hour(24-Pac)']== 21),(DF['Hour(24-Pac)']== 22),(DF['Hour(24-Pac)']== 23),
    
]

if Timezone == 'West':

    pac=['0am-pt','1am-pt','2am-pt','3am-pt','4am-pt','5am-pt',
        '6am-pt','7am-pt','8am-pt','9am-pt','10am-pt','11am-pt',
        '12pm-pt','1pm-pt','2pm-pt','3pm-pt','4pm-pt','5pm-pt',
        '6pm-pt','7pm-pt','8pm-pt','9pm-pt','10pm-pt','11pm-pt',]

    east=['3am-et','4am-et','5am-et','6am-et','7am-et','8am-et',
          '9am-et','10am-et','11am-et','12pm-et','1pm-et','2pm-et',
          '3pm-et','4pm-et','5pm-et','6pm-et','7pm-et','8pm-et',
          '9pm-et','10pm-et','11pm-et','0am-et','1am-et','2am-et']
else:
    pac=['9pm-pt','10pm-pt','11pm-pt','0am-pt','1am-pt','2am-pt',
         '3am-pt','4am-pt','5am-pt','6am-pt','7am-pt','8am-pt',
         '9am-pt','10am-pt','11am-pt','12pm-pt','1pm-pt','2pm-pt',
         '3pm-pt','4pm-pt','5pm-pt','6pm-pt','7pm-pt','8pm-pt']

    east=['0am-et','1am-et','2am-et','3am-et','4am-et','5am-et',
          '6am-et','7am-et','8am-et','9am-et','10am-et','11am-et',
          '12pm-et','1pm-et','2pm-et','3pm-et','4pm-et','5pm-et',
          '6pm-et','7pm-et','8pm-et','9pm-et','10pm-et','11pm-et']


DF['Pacific_Time']=np.select(Con4,pac,default=np.nan)
DF['East_Time']=np.select(Con4,east,default=np.nan)

In [56]:
DF= DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday(num)','Weekday','Hour(24-Pac)','Pacific_Time',
              'East_Time','Series/Film','Title','Daypart','Minutes per Minute','Sessions per Minute','Session Duration','Year','Month')]

In [57]:
DF

,Content ID,TS,TS End,Length(min) in the Hour,Day,Weekday(num),Weekday,Hour(24-Pac),Pacific_Time,East_Time,Series/Film,Title,Daypart,Minutes per Minute,Sessions per Minute,Session Duration,Year,Month
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,56.852,1,4,Fri,0,0am-pt,3am-et,Series,World of Weird,Overnight,3.733333,0.250000,14.933333,2021,10
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,4.000,1,4,Fri,0,0am-pt,3am-et,Series,World of Weird,Overnight,3.733333,0.250000,14.933333,2021,10
2,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,6.000,1,4,Fri,1,1am-pt,4am-et,Series,World of Weird,Overnight,3.250000,0.250000,13.000000,2021,10
3,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,10.000,1,4,Fri,2,2am-pt,5am-et,Series,World of Weird,Overnight,5.650000,0.300000,18.833333,2021,10
4,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,14.000,1,4,Fri,3,3am-pt,6am-et,Series,17 Kids and Counting,Weekday Morning,6.266667,0.250000,25.066667,2021,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9436,1000000002847,2022-08-31 18:14:15.110,2022-08-31 19:10:47.240,10.000,31,2,Wed,19,7pm-pt,10pm-et,Series,Hoarders Get Your House In Order,Prime,17.891500,5.516667,3.243172,2022,8
9437,1000000002848,2022-08-31 19:10:47.240,2022-08-31 20:07:23.160,7.000,31,2,Wed,20,8pm-pt,11pm-et,Series,Hoarders Get Your House In Order,Late Night,11.012000,4.600000,2.393913,2022,8
9438,1000000005027,2022-08-31 20:07:23.160,2022-08-31 21:04:23.030,4.000,31,2,Wed,21,9pm-pt,0am-et,Series,Tattoo Fixers on Holiday - Clean,Late Night,15.721667,3.650000,4.307306,2022,8
9440,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,55.000,31,2,Wed,22,10pm-pt,1am-et,Series,Tattoo Fixers on Holiday - Clean,Late Night,9.242333,2.400000,3.850972,2022,8


# Building Confidence Intervals

In [58]:
DF.sort_values(by=['TS','Hour(24-Pac)'], inplace=True)
DF=DF.reset_index()

In [59]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Year'] = DF['TS End'].dt.year

In [60]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==1),'Year'] = DF['TS End'].dt.year

In [61]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Day'] = DF['TS End'].dt.day
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Month'] = DF['TS End'].dt.month
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==2),'Year'] = DF['TS End'].dt.year

In [62]:
DF.loc[(DF['TS'].dt.hour != DF['TS End'].dt.hour) & (DF['Hour(24-Pac)']==0),'Weekday(num)'] = DF['TS End'].dt.weekday

In [63]:
Condwk=[
    DF['Weekday(num)']== 0,DF['Weekday(num)']== 1,DF['Weekday(num)']== 2,DF['Weekday(num)']== 3,
    DF['Weekday(num)']== 4,DF['Weekday(num)']== 5,DF['Weekday(num)']== 6   
]

wkdays= ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

DF['Weekday(name)']= np.select(Condwk,wkdays, default=np.nan)

In [64]:
if Timezone == 'West':
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 13) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 13),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 7) & (DF['Hour(24-Pac)'] >= 3),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 17) & (DF['Hour(24-Pac)'] >= 7),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 17),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 3) | (DF['Hour(24-Pac)'] >= 23)),

    ]
else:
    Conditions1 =[
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 9) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 16) & (DF['Hour(24-Pac)'] >= 9),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 16),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] < 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] < 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 10) & (DF['Hour(24-Pac)'] >= 6),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 20) & (DF['Hour(24-Pac)'] >= 10),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 23) & (DF['Hour(24-Pac)'] >= 20),
        (DF['Weekday(num)'] >= 5) & ((DF['Hour(24-Pac)'] < 2) | (DF['Hour(24-Pac)'] >= 23)),
        (DF['Weekday(num)'] >= 5) & (DF['Hour(24-Pac)'] < 6) & (DF['Hour(24-Pac)'] >= 2),

    ]
    

Daypart1 = ['Weekday Morning','Weekday Day', 'Weekday Fringe','Prime', 'Late Night', 'Overnight',
           'Weekend Morning', 'Weekend Day', 'Prime (wknd)', 'Late Night (wknd)', 'Overnight (wknd)'
         ]

DF['Daypart1']= np.select(Conditions1, Daypart1, default= np.nan)

In [65]:
DF.rename(columns={'Daypart':'no','Daypart1':'Daypart'}, inplace=True)

In [66]:
DF=DF.loc[:,('Content ID','TS','TS End','Length(min) in the Hour','Day','Weekday(num)','Weekday(name)','Hour(24-Pac)',
            'Pacific_Time','East_Time','Series/Film','Title','Daypart','Minutes per Minute',
             'Sessions per Minute','Session Duration','Year','Month')]

In [67]:
Cond=[
    (DF['Weekday(num)']== 5) | (DF['Weekday(num)']==6),
    (DF['Weekday(num)']== 0)|(DF['Weekday(num)']==1)|(DF['Weekday(num)']== 2)|(DF['Weekday(num)']==3)|
    (DF['Weekday(num)']== 4)
]

Values=['Weekend','Weekday']

DF['Weekdays/Weekends']= np.select(Cond,Values)

# What is the Average

In [68]:
HR=DF.groupby(['Year','Month','Weekdays/Weekends','Day','Hour(24-Pac)']).mean().reset_index()

In [69]:
HR.head(10)

,Year,Month,Weekdays/Weekends,Day,Hour(24-Pac),Length(min) in the Hour,Weekday(num),Minutes per Minute,Sessions per Minute,Session Duration
0,2021,10,Weekday,1,0,30.426,4,3.733333,0.250000,14.933333
1,2021,10,Weekday,1,1,30.000,4,3.250000,0.250000,13.000000
2,2021,10,Weekday,1,2,30.000,4,5.650000,0.300000,18.833333
3,2021,10,Weekday,1,3,30.000,4,6.266667,0.250000,25.066667
4,2021,10,Weekday,1,4,30.000,4,5.666667,0.166667,34.000000
5,2021,10,Weekday,1,5,30.000,4,6.133333,0.350000,17.523810
6,2021,10,Weekday,1,6,30.000,4,9.266667,0.516667,17.935484
7,2021,10,Weekday,1,7,30.000,4,9.183333,0.250000,36.733333
8,2021,10,Weekday,1,8,30.000,4,8.100000,0.350000,23.142857
9,2021,10,Weekday,1,9,30.000,4,8.466667,0.316667,26.736842


In [70]:
Hourly_AVG= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).mean().reset_index()
Hourly_STD= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).std().reset_index()
Hourly_Count= HR.loc[:,('Minutes per Minute','Weekdays/Weekends','Hour(24-Pac)','Year','Month')].groupby(['Year','Month','Weekdays/Weekends',
                                        'Hour(24-Pac)']).count().reset_index()

In [71]:
Hourly_AVG.rename(columns={'Minutes per Minute':'Hourly AVG'},inplace=True)
Hourly_STD.rename(columns={'Minutes per Minute':'Hourly STD'},inplace=True)
Hourly_Count.rename(columns={'Minutes per Minute':'Hourly Count'},inplace=True)

In [72]:
DF= pd.merge(DF,Hourly_AVG, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_STD, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')
DF= pd.merge(DF,Hourly_Count, on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')

# Adjust T-Value

In [73]:
data={'Count':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30],
      'Tvalue':[12.706,4.303,3.182,2.776,2.571,2.447,2.365,2.306,2.262,2.228,2.201,2.179,2.16,2.145,2.131,2.12,2.11,2.101,2.093,2.086,2.08,2.074,2.069,2.064,2.06,2.056,2.052,2.048,2.045,2.042]   
}
T_values=pd.DataFrame(data)

In [74]:
DF=pd.merge(DF,T_values, left_on='Hourly Count',right_on='Count', how='left')

In [75]:
DF['Upper CI(95%)']= DF['Hourly AVG']+(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))
DF['Lower CI(95%)']= DF['Hourly AVG']-(DF['Tvalue']* (DF['Hourly STD']/(DF['Hourly Count']**(1/2))))

In [76]:
Con1=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%)'])     
]

Values1= ['Above','Below','Neutral']

DF['Above/Below/Neutral']=np.select(Con1,Values1, default=np.nan)

In [77]:
Con1=[
    (DF['Above/Below/Neutral']== 'Above'), (DF['Above/Below/Neutral']== 'Below'), (DF['Above/Below/Neutral']== 'Neutral'),
]

Values1=[(DF['Minutes per Minute']-DF['Upper CI(95%)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%)']),
         0]
DF['Minutes Diff CI']= np.select(Con1,Values1, default=np.nan)

In [78]:
DF['%Minutes Diff CI']=(DF['Minutes Diff CI']/DF['Hourly AVG'])

In [79]:
DF['Minutes Diff']= DF['Minutes per Minute']-DF['Hourly AVG']
DF['%Minutes Diff']= DF['Minutes Diff']/DF['Hourly AVG']
DF['Total Minutes in Hour']= DF['Minutes per Minute']*DF['Length(min) in the Hour']
DF['Total Sessions in Hour']= DF['Sessions per Minute']*DF['Length(min) in the Hour']

del DF['Count']

In [80]:
DF['TS_test']= DF['TS'].dt.day

In [81]:
CondM=[
    (DF['Month']== 1) | (DF['Month']==2) | (DF['Month']==3),
    (DF['Month']== 4) | (DF['Month']==5) | (DF['Month']==6),
    (DF['Month']== 7) | (DF['Month']==8) | (DF['Month']==9),
    (DF['Month']== 10) | (DF['Month']==11) | (DF['Month']==12)
]

ValuesQ=['Q1','Q2','Q3','Q4']

DF['Quarter']= np.select(CondM,ValuesQ)

In [82]:
Condm=[
    (DF['Month']== 1) , (DF['Month']==2) , (DF['Month']==3),
    (DF['Month']== 4) , (DF['Month']==5) , (DF['Month']==6),
    (DF['Month']== 7) , (DF['Month']==8) , (DF['Month']==9),
    (DF['Month']== 10) , (DF['Month']==11) , (DF['Month']==12)
]

Valuesq=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

DF['Month(name)']= np.select(Condm,Valuesq)

In [83]:
DF['M_Y']= DF['Month'].astype(str)+'-'+ DF['Year'].astype(str)

In [84]:
DF.loc[(DF['Day']==1), ('Day','Month')].drop_duplicates()

,Day,Month
0,1,10
1687,1,11
3355,1,12
4990,1,1
6559,1,2
7976,1,3
9502,1,4
10977,1,5
12485,1,6
13927,1,7


# Created Day(Adj.)

In [85]:
M_Y_list = DF['M_Y'].unique()

In [86]:
M_Y_list

array(['10-2021', '11-2021', '12-2021', '1-2022', '2-2022', '3-2022',
       '4-2022', '5-2022', '6-2022', '7-2022', '8-2022'], dtype=object)

In [87]:
M_Ydf= pd.DataFrame()
    
for i in M_Y_list:
    MY_DF= DF.loc[DF['M_Y']== i,('M_Y','Day','Weekday(num)')].drop_duplicates()
    MY_DF['Day (adj.)']= MY_DF.loc[:,'Day']
    if (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==1).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 1
    elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==2).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 2
    elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==3).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 3
    elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==4).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 4
    elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==5).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 5
    elif (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==6).bool():
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 6
    else:
        (MY_DF.loc[MY_DF['Day'] == 1,'Weekday(num)']==0).bool()
        MY_DF['Day (adj.)'] = MY_DF['Day (adj.)'] + 0
        
        
    M_Ydf = M_Ydf.append(MY_DF)

In [88]:
DF = pd.merge(DF,M_Ydf.loc[:,('M_Y','Day','Day (adj.)')], on = ('M_Y','Day'), how='left')

In [89]:
Cond_wknd = [ (DF['Weekdays/Weekends'] == 'Weekday') , (DF['Weekdays/Weekends'] == 'Weekend') ]

Values_wknd = ['Wk','Wknd']

DF['Wk/Wknd'] = np.select(Cond_wknd,Values_wknd)

In [90]:
if Timezone == 'West':
    DF['TS_East'] = DF['TS'] + timedelta(hours=3)
else:
    DF['TS_East'] = ''

In [91]:
DF['Title']=DF.groupby('Content ID')['Title'].transform('last')

In [92]:
DF=DF.loc[:,('Content ID','TS','TS End','TS_East','M_Y','Year','Quarter','Month','Month(name)','Day','Day (adj.)','Weekday(name)','Hour(24-Pac)','Pacific_Time','East_Time','Weekdays/Weekends','Wk/Wknd',
             'Daypart','Series/Film','Title','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour','Minutes per Minute','Sessions per Minute',
            'Session Duration','Minutes Diff','%Minutes Diff','Hourly AVG','Hourly STD','Upper CI(95%)','Lower CI(95%)','Minutes Diff CI','%Minutes Diff CI','Above/Below/Neutral')]

In [93]:
# LAST DF_______________LAST DF______________________________

DF.head(10)

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,2021-10-01 03:00:00.000,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,56.852,212.247467,14.213000,3.733333,0.250000,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,4.000,14.933333,1.000000,3.733333,0.250000,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below
2,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,54.000,175.500000,13.500000,3.250000,0.250000,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below
3,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,6.000,19.500000,1.500000,3.250000,0.250000,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below
4,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,50.000,282.500000,15.000000,5.650000,0.300000,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral
5,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,2021-10-01 05:50:56.240,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,10.000,56.500000,3.000000,5.650000,0.300000,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral
6,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,2021-10-01 05:50:56.240,10-2021,2021,Q4,10,Oct,1,5,Fri,3,3am-pt,6am-et,Weekday,Wk,Weekday Morning,Series,World of Weird,46.000,288.266667,11.500000,6.266667,0.250000,25.066667,1.579365,0.336945,4.687302,2.217837,5.693963,3.680640,0.572704,0.122182,Above
7,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,2021-10-01 06:46:39.050,10-2021,2021,Q4,10,Oct,1,5,Fri,3,3am-pt,6am-et,Weekday,Wk,Weekday Morning,Series,17 Kids and Counting,14.000,87.733333,3.500000,6.266667,0.250000,25.066667,1.579365,0.336945,4.687302,2.217837,5.693963,3.680640,0.572704,0.122182,Above
8,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,2021-10-01 06:46:39.050,10-2021,2021,Q4,10,Oct,1,5,Fri,4,4am-pt,7am-et,Weekday,Wk,Weekday Morning,Series,17 Kids and Counting,44.000,249.333333,7.333333,5.666667,0.166667,34.000000,1.434921,0.339085,4.231746,1.669156,4.989364,3.474128,0.677302,0.160053,Above
9,CEG10028441,2021-10-01 04:44:47.280,2021-10-01 05:40:33.090,2021-10-01 07:44:47.280,10-2021,2021,Q4,10,Oct,1,5,Fri,4,4am-pt,7am-et,Weekday,Wk,Weekday Morning,Series,18 Kids and Counting,16.000,90.666667,2.666667,5.666667,0.166667,34.000000,1.434921,0.339085,4.231746,1.669156,4.989364,3.474128,0.677302,0.160053,Above


# Adding 3 Hour Avg Interval

In [94]:
Amagi.head(10)

,Date,Platform/Channel,Session Count,Total Viewership Minutes,Unique Viewers,Avg Session Count,Avg Session Duration Per Session,Avg Session Duration Per Viewer,Platform,Channel,Hour,Day,Year,Month
21,2021-10-01 00:00:00,Cinedigm_Soreal_TCL,1,0.0,1,NaN,NaN,NaN,TCL,SoReal,0,1,2021,10
61,2021-10-01 00:00:00,soreal_littlstar,1,60.0,1,NaN,NaN,NaN,Littlstar,SoReal,0,1,2021,10
62,2021-10-01 00:00:00,SoReal Plex,13,164.0,11,NaN,NaN,NaN,Plex,SoReal,0,1,2021,10
120,2021-10-01 01:00:00,soreal_littlstar,1,60.0,1,NaN,NaN,NaN,Littlstar,SoReal,1,1,2021,10
121,2021-10-01 01:00:00,SoReal Plex,12,128.0,12,NaN,NaN,NaN,Plex,SoReal,1,1,2021,10
122,2021-10-01 01:00:00,So Real Stremium US,2,7.0,2,NaN,NaN,NaN,Stremium,SoReal,1,1,2021,10
137,2021-10-01 02:00:00,Cinedigm_SoReal_KlowdTV,1,0.0,1,NaN,NaN,NaN,Klowd,SoReal,2,1,2021,10
176,2021-10-01 02:00:00,soreal_littlstar,1,60.0,1,NaN,NaN,NaN,Littlstar,SoReal,2,1,2021,10
177,2021-10-01 02:00:00,SoReal Plex,15,279.0,13,NaN,NaN,NaN,Plex,SoReal,2,1,2021,10
178,2021-10-01 02:00:00,So Real Stremium US,1,0.0,1,NaN,NaN,NaN,Stremium,SoReal,2,1,2021,10


In [95]:
Avg3=Amagi.loc[:,:]
Avg3['Wn']=Avg3['Date'].dt.weekday

<ipython-input-95-e8e758263c4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avg3['Wn']=Avg3['Date'].dt.weekday


In [96]:
condweek=[
    (Avg3['Wn']== 0) | (Avg3['Wn']==1) | (Avg3['Wn']== 2) | (Avg3['Wn']==3) | (Avg3['Wn']== 4), 
    (Avg3['Wn']== 5) | (Avg3['Wn']==6) 
]
results=('Weekday','Weekend')

Avg3['week/weekend']=np.select(condweek,results, default=np.nan)

<ipython-input-96-0fb7175f0b20>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Avg3['week/weekend']=np.select(condweek,results, default=np.nan)


In [97]:
Avg3=Amagi.groupby(by=['Year','Month','week/weekend','Day','Hour'])['Total Viewership Minutes'].agg(sum).reset_index()

In [98]:
Avg3['Min/Min']=Avg3['Total Viewership Minutes']/60
Avg3=Avg3.loc[:,('Year','Month','week/weekend','Day','Hour','Min/Min')]

In [99]:
Avg3

,Year,Month,week/weekend,Day,Hour,Min/Min
0,2021,10,Weekday,1,0,3.733333
1,2021,10,Weekday,1,1,3.250000
2,2021,10,Weekday,1,2,5.650000
3,2021,10,Weekday,1,3,6.266667
4,2021,10,Weekday,1,4,5.666667
...,...,...,...,...,...,...
8021,2022,8,Weekend,28,19,26.241167
8022,2022,8,Weekend,28,20,14.576000
8023,2022,8,Weekend,28,21,16.283667
8024,2022,8,Weekend,28,22,16.840833


In [100]:
def Year_Month(y,m):
    #Weekdays
    def Mean_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
        return HourwM

    def STD_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
        return HourwM

    def Cnt_Wk(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
        return HourwM

    #Weekends
    def Mean_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
        return HourwM

    def STD_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
        return HourwM

    def Cnt_Wknd(H):
        HourwM = Avg3.loc[((Avg3['Hour']== H-1) | (Avg3['Hour']== H) | (Avg3['Hour']== H+1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
        return HourwM

        #Exceptions___23 and 0 for Month 1 Year 2022_________________________________________________________

       #Weekdays
    Hour23wM = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour23wS = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour23wC = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
       #Weekends
    Hour23wkM = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour23wkS = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour23wkC = Avg3.loc[((Avg3['Hour']== 22) | (Avg3['Hour']== 23) | (Avg3['Hour']== 0)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()

       #Weekdays
    Hour0wM = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour0wS = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour0wC = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekday') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
       #Weekends
    Hour0wkM = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].mean()
    Hour0wkS = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].std()
    Hour0wkC = Avg3.loc[((Avg3['Hour']== 23) | (Avg3['Hour']== 0) | (Avg3['Hour']== 1)) & (Avg3['week/weekend']== 'Weekend') & (Avg3['Year']==y) & (Avg3['Month']==m), ('Min/Min')].count()
    
    #Creating TABLE________________________________________________________
    
    H3Data={'Year':[y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y,y],
        'Month':[m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m,m],
    
        'Weekdays/Weekends':['Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday',
                             'Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday','Weekday',
                  'Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend',
                             'Weekend','Weekend','Weekend','Weekend','Weekend','Weekend','Weekend'],
        
        'Hour(24-Pac)':[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,     0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],
        
        'Mean (H3)':[Hour0wM,Mean_Wk(1),Mean_Wk(2),Mean_Wk(3),Mean_Wk(4),Mean_Wk(5),Mean_Wk(6),Mean_Wk(7),Mean_Wk(8),Mean_Wk(9),Mean_Wk(10),Mean_Wk(11),
                Mean_Wk(12),Mean_Wk(13),Mean_Wk(14),Mean_Wk(15),Mean_Wk(16),Mean_Wk(17),Mean_Wk(18),Mean_Wk(19),Mean_Wk(20),Mean_Wk(21),Mean_Wk(22),Hour23wM,
               
                Hour0wkM,Mean_Wknd(1),Mean_Wknd(2),Mean_Wknd(3),Mean_Wknd(4),Mean_Wknd(5),Mean_Wknd(6),Mean_Wknd(7),Mean_Wknd(8),Mean_Wknd(9),Mean_Wknd(10),Mean_Wknd(11),
                Mean_Wknd(12),Mean_Wknd(13),Mean_Wknd(14),Mean_Wknd(15),Mean_Wknd(16),Mean_Wknd(17),Mean_Wknd(18),Mean_Wknd(19),Mean_Wknd(20),Mean_Wknd(21),Mean_Wknd(22),Hour23wkM],
        
        
        'STD (H3)':[Hour0wS,STD_Wk(1),STD_Wk(2),STD_Wk(3),STD_Wk(4),STD_Wk(5),STD_Wk(6),STD_Wk(7),STD_Wk(8),STD_Wk(9),STD_Wk(10),STD_Wk(11),
                STD_Wk(12),STD_Wk(13),STD_Wk(14),STD_Wk(15),STD_Wk(16),STD_Wk(17),STD_Wk(18),STD_Wk(19),STD_Wk(20),STD_Wk(21),STD_Wk(22),Hour23wS,
               
                Hour0wkS,STD_Wknd(1),STD_Wknd(2),STD_Wknd(3),STD_Wknd(4),STD_Wknd(5),STD_Wknd(6),STD_Wknd(7),STD_Wknd(8),STD_Wknd(9),STD_Wknd(10),STD_Wknd(11),
                STD_Wknd(12),STD_Wknd(13),STD_Wknd(14),STD_Wknd(15),STD_Wknd(16),STD_Wknd(17),STD_Wknd(18),STD_Wknd(19),STD_Wknd(20),STD_Wknd(21),STD_Wknd(22),Hour23wkS],
        
        
        'Count (H3)':[Hour0wC,Cnt_Wk(1),Cnt_Wk(2),Cnt_Wk(3),Cnt_Wk(4),Cnt_Wk(5),Cnt_Wk(6),Cnt_Wk(7),Cnt_Wk(8),Cnt_Wk(9),Cnt_Wk(10),Cnt_Wk(11),
                Cnt_Wk(12),Cnt_Wk(13),Cnt_Wk(14),Cnt_Wk(15),Cnt_Wk(16),Cnt_Wk(17),Cnt_Wk(18),Cnt_Wk(19),Cnt_Wk(20),Cnt_Wk(21),Cnt_Wk(22),Hour23wC,
               
                Hour0wkC,Cnt_Wknd(1),Cnt_Wknd(2),Cnt_Wknd(3),Cnt_Wknd(4),Cnt_Wknd(5),Cnt_Wknd(6),Cnt_Wknd(7),Cnt_Wknd(8),Cnt_Wknd(9),Cnt_Wknd(10),Cnt_Wknd(11),
                Cnt_Wknd(12),Cnt_Wknd(13),Cnt_Wknd(14),Cnt_Wknd(15),Cnt_Wknd(16),Cnt_Wknd(17),Cnt_Wknd(18),Cnt_Wknd(19),Cnt_Wknd(20),Cnt_Wknd(21),Cnt_Wknd(22),Hour23wkC]
    }

    return pd.DataFrame(H3Data)


In [101]:
UY=Avg3['Year'].unique()
UM=Avg3['Month'].unique()

In [102]:
DataH3=pd.DataFrame(columns=['Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','STD (H3)','Count (H3)'])
for i,j in [(i,j) for i in UY for j in UM]:
    H3=pd.DataFrame(Year_Month(i,j))
    DataH3=DataH3.append(H3)

In [103]:
DataH3 = DataH3.loc[DataH3['Count (H3)']!=0, :]

In [104]:
DataH3=pd.merge(DataH3,T_values, left_on='Count (H3)', right_on='Count', how='left')
DataH3['Z-value']= [1.96 if ele =='Weekday' else DataH3['Tvalue'] for ele in DataH3['Weekdays/Weekends']]

In [105]:
Cond33= [
    DataH3['Weekdays/Weekends']=='Weekday',
    DataH3['Weekdays/Weekends']=='Weekend'
]
Choices= [ DataH3['Z-value'], DataH3['Tvalue']]

DataH3['Value']= np.select(Cond33, Choices, default=np.nan)

In [106]:
DataH3['Upper CI(95%) (3H)']= DataH3['Mean (H3)']+(DataH3['Value']* (DataH3['STD (H3)']/(DataH3['Count (H3)']**(1/2))))
DataH3['Lower CI(95%) (3H)']= DataH3['Mean (H3)']-(DataH3['Value']* (DataH3['STD (H3)']/(DataH3['Count (H3)']**(1/2))))

In [107]:
DataH3= DataH3.loc[:,('Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','STD (H3)','Count (H3)','Value','Upper CI(95%) (3H)','Lower CI(95%) (3H)')]

In [108]:
DF= pd.merge(DF, DataH3.loc[:,('Year','Month','Weekdays/Weekends','Hour(24-Pac)','Mean (H3)','Upper CI(95%) (3H)','Lower CI(95%) (3H)')], on=['Year','Month','Weekdays/Weekends','Hour(24-Pac)'], how='left')

In [109]:
Con13=[
    (DF['Minutes per Minute'] > DF['Upper CI(95%) (3H)']),
    (DF['Minutes per Minute'] < DF['Lower CI(95%) (3H)']),
    (DF['Minutes per Minute'] <= DF['Upper CI(95%) (3H)']) & (DF['Minutes per Minute'] >= DF['Lower CI(95%) (3H)'])     
]

Values13= ['Above','Below','Neutral']

DF['Above/Below/Neutral (3H)']=np.select(Con13,Values13, default=np.nan)

In [110]:
Con14=[
    (DF['Above/Below/Neutral (3H)']== 'Above'), (DF['Above/Below/Neutral (3H)']== 'Below'), (DF['Above/Below/Neutral (3H)']== 'Neutral'),
]

Values14=[(DF['Minutes per Minute']-DF['Upper CI(95%) (3H)']),
         (DF['Minutes per Minute']-DF['Lower CI(95%) (3H)']),
         0]
DF['Minutes Diff CI (3H)']= np.select(Con14,Values14, default=np.nan)

In [111]:
DF['%Minutes Diff CI (3H)']=(DF['Minutes Diff CI (3H)']/DF['Mean (H3)'])

In [112]:
DF['Upper CI(95%) (3H)']= DF['Upper CI(95%) (3H)'].astype(float)
DF['Lower CI(95%) (3H)']= DF['Lower CI(95%) (3H)'].astype(float)
DF['Minutes Diff CI (3H)']= DF['Minutes Diff CI (3H)'].astype(float)
DF['%Minutes Diff CI (3H)']= DF['%Minutes Diff CI (3H)'].astype(float)

# Daypart Average Using DF Table

In [113]:
DFdpavg=DF.loc[:,('Year','Month','Daypart','Minutes per Minute')]

In [114]:
DFdpavg= DFdpavg.groupby(by=['Year','Month','Daypart']).mean().reset_index()

In [115]:
DFdpavg.rename(columns={'Minutes per Minute':'Mean (Month DP)'},inplace=True)

In [116]:
DF= pd.merge(DF,DFdpavg, on=['Year','Month','Daypart'], how='left')

In [117]:
DF['%Minutes Diff (Month DP)']= (DF['Minutes per Minute']-DF['Mean (Month DP)'])/DF['Mean (Month DP)']

In [118]:
DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,2021-10-01 03:00:00.000,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,56.8520,212.247467,14.213000,3.733333,0.25,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,4.0000,14.933333,1.000000,3.733333,0.25,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
2,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,54.0000,175.500000,13.500000,3.250000,0.25,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
3,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,6.0000,19.500000,1.500000,3.250000,0.25,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
4,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,50.0000,282.500000,15.000000,5.650000,0.30,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral,4.936508,5.461814,4.411202,Above,0.188186,0.038121,5.076000,0.113081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16934,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,2022-09-01 00:04:23.030,8-2022,2022,Q3,8,Aug,31,31,Wed,21,9pm-pt,0am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,54.9205,863.441794,200.459825,15.721667,3.65,4.307306,-0.435043,-0.026926,16.156710,4.110143,17.929893,14.383527,0.000000,0.000000,Neutral,15.754947,16.849181,14.660712,Neutral,0.000000,0.000000,15.703523,0.001155
16935,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,2022-09-01 00:59:18.260,8-2022,2022,Q3,8,Aug,31,31,Wed,21,9pm-pt,0am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,1.0000,15.721667,3.650000,15.721667,3.65,4.307306,-0.435043,-0.026926,16.156710,4.110143,17.929893,14.383527,0.000000,0.000000,Neutral,15.754947,16.849181,14.660712,Neutral,0.000000,0.000000,15.703523,0.001155
16936,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,2022-09-01 00:59:18.260,8-2022,2022,Q3,8,Aug,31,31,Wed,22,10pm-pt,1am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,55.0000,508.328333,132.000000,9.242333,2.40,3.850972,-3.943391,-0.299065,13.185725,3.699559,14.781775,11.589674,-2.347341,-0.178021,Below,13.029534,14.078917,11.980151,Below,-2.737818,-0.210124,15.703523,-0.411448
16937,1000000005030,2022-08-31 22:55:51.240,2022-08-31 23:59:59.270,2022-09-01 01

# Adding TOTALs

In [119]:
TotS= DF.loc[:,('Title','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour')]
TotS=TotS.groupby('Title').agg(sum).reset_index()
TotS['Session Duration']= TotS['Total Minutes in Hour']/TotS['Total Sessions in Hour']

TotS.rename(columns={'Length(min) in the Hour':'Total Min Aired','Total Minutes in Hour':'Total Min Viewed','Total Sessions in Hour':'Total Sessions'}, inplace=True)

In [120]:
TotSD= DF.loc[:,('Title','Daypart','Length(min) in the Hour','Total Minutes in Hour','Total Sessions in Hour')]
TotSD=TotSD.groupby(by=['Title','Daypart']).agg(sum).reset_index()
TotSD['Session Duration']= TotSD['Total Minutes in Hour']/TotSD['Total Sessions in Hour']

TotSD.rename(columns={'Length(min) in the Hour':'Total Min Aired','Total Minutes in Hour':'Total Min Viewed','Total Sessions in Hour':'Total Sessions'}, inplace=True)

# Content Summary

In [121]:
CS=DF.loc[:,('Title','Series/Film','Above/Below/Neutral (3H)','Content ID')]

In [122]:
CS=CS.pivot_table(index= ['Series/Film','Title'], values='Content ID', columns='Above/Below/Neutral (3H)', aggfunc=pd.Series.count)
CS=CS.reset_index()

In [123]:
CS['Above']=CS['Above'].fillna(0)
CS['Below']=CS['Below'].fillna(0)
CS['Neutral']=CS['Neutral'].fillna(0)


CS['Count']= (CS['Above']+ CS['Below']+ CS['Neutral'])
CS['%Below CI']= CS['Below']/CS['Count']
CS['%Inside CI']= CS['Neutral']/CS['Count']
CS['%Above CI']= CS['Above']/CS['Count']
CS['Score']= ((CS['Above']**2)-(CS['Below']**2))/CS['Count']

In [124]:
CS1=DF.loc[:,('Title','Series/Film','%Minutes Diff CI','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Minutes per Minute')]
CS1= CS1.groupby('Title').mean().reset_index()
CSsumm= pd.merge(CS,CS1, on='Title', how='left')

In [125]:
#Combined Score

CondSC=[
    (CSsumm['Score']> 0) & (CSsumm['%Minutes Diff CI (3H)']> 0),
     (CSsumm['Score']> 0) & (CSsumm['%Minutes Diff CI (3H)']< 0),
     (CSsumm['Score']== 0) | (CSsumm['%Minutes Diff CI (3H)']== 0),
     (CSsumm['Score']< 0) & (CSsumm['%Minutes Diff CI (3H)']< 0)   
]

ValuesSC=[CSsumm['Score']*CSsumm['%Minutes Diff CI (3H)'],
         0,
         0,
         (CSsumm['Score']*CSsumm['%Minutes Diff CI (3H)'])*-1
        ]

CSsumm['Combined Score']= np.select(CondSC,ValuesSC)

In [126]:
CSsumm= pd.merge(CSsumm,TotS, on='Title', how='left')

In [127]:
CSsumm= CSsumm.loc[:,('Series/Film','Title','Count','Above','Neutral','Below','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Combined Score',
                      'Total Min Aired','Total Min Viewed','Total Sessions','Session Duration')].sort_values('Combined Score', ascending=False)

In [128]:
CSsumm.head(10)

,Series/Film,Title,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
42,Series,Tattoo Fixers - Clean,1995.0,1047.0,418.0,530.0,18.416657,0.116176,0.175738,47.478403,60656.952500,1.112132e+06,196972.767744,5.646119
26,Series,Hoarders Get Your House In Order,911.0,526.0,179.0,206.0,20.153264,0.141927,0.205703,36.492856,26651.207833,5.413802e+05,121224.132022,4.465944
27,Series,Hoarders SOS,1163.0,542.0,275.0,346.0,19.616242,0.079117,0.134280,11.840254,35041.961333,6.893791e+05,142650.126981,4.832657
38,Series,Perfect Vagina,333.0,167.0,80.0,86.0,11.751974,0.125007,0.159681,7.693014,10386.000000,1.262600e+05,22006.566667,5.737379
30,Series,"Kinky Sex Survey, The",292.0,139.0,81.0,72.0,11.757354,0.110840,0.134191,5.366258,9751.012000,1.158808e+05,22040.084211,5.257732
20,Series,Danger Beach,302.0,148.0,73.0,81.0,7.543424,0.061683,0.085913,3.133804,5648.351667,4.259969e+04,4107.894361,10.370201
12,Series,Air Rescue,775.0,324.0,200.0,251.0,9.322344,0.046717,0.048005,2.530257,14406.285833,1.345945e+05,19315.639242,6.968161
45,Series,The Cops - Clean,136.0,67.0,43.0,26.0,14.006143,0.081926,0.074172,2.296929,3807.770333,5.537587e+04,18683.417917,2.963905
11,Series,Addicted to Boob Jobs,342.0,145.0,85.0,112.0,10.910143,0.077203,0.083245,1.914487,10263.000000,1.120816e+05,19632.800000,5.708895
32,Series,"Love Me, Love My Doll",259.0,104.0,65.0,90.0,9.682793,0.050567,0.053409,0.530270,9704.207167,9.214276e+04,12088.315750,7.622464


# Content Summary with Dayparts

In [129]:
CSD=DF.loc[:,('Title','Daypart','Series/Film','Above/Below/Neutral (3H)','Content ID')]

In [130]:
CSD=CSD.pivot_table(index= ['Series/Film','Title','Daypart'], values='Content ID', columns='Above/Below/Neutral (3H)', aggfunc=pd.Series.count)
CSD=CSD.reset_index()

In [131]:
CSD['Above']=CSD['Above'].fillna(0)
CSD['Below']=CSD['Below'].fillna(0)
CSD['Neutral']=CSD['Neutral'].fillna(0)


CSD['Count']= (CSD['Above']+ CSD['Below']+ CSD['Neutral'])
CSD['%Below CI']= CSD['Below']/CSD['Count']
CSD['%Inside CI']= CSD['Neutral']/CSD['Count']
CSD['%Above CI']= CSD['Above']/CSD['Count']
CSD['Score']= ((CSD['Above']**2)-(CSD['Below']**2))/CSD['Count']

In [132]:
CSD1=DF.loc[:,('Title','Series/Film','Daypart','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)')]
CSD1= CSD1.groupby(['Title','Daypart']).mean().reset_index()
CSDsumm= pd.merge(CSD,CSD1, on=['Title','Daypart'], how='left')

In [133]:
CondSCD=[
    (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI (3H)']> 0),
     (CSDsumm['Score']> 0) & (CSDsumm['%Minutes Diff CI (3H)']< 0),
     (CSDsumm['Score']== 0) | (CSDsumm['%Minutes Diff CI (3H)']== 0),
     (CSDsumm['Score']< 0) & (CSDsumm['%Minutes Diff CI (3H)']< 0)   
]

ValuesSCD=[CSDsumm['Score']*CSDsumm['%Minutes Diff CI (3H)'],
         0,
         0,
         (CSDsumm['Score']*CSDsumm['%Minutes Diff CI (3H)'])*-1
        ]

CSDsumm['Combined Score']= np.select(CondSCD,ValuesSCD)

In [134]:
CSDsumm= pd.merge(CSDsumm,TotSD, on=['Title','Daypart'], how='left')

In [135]:
CSDsumm=CSDsumm.loc[:,('Series/Film','Title','Daypart','Count','Above','Neutral','Below','Minutes per Minute','%Minutes Diff CI (3H)','%Minutes Diff (Month DP)','Combined Score',
                      'Total Min Aired','Total Min Viewed','Total Sessions','Session Duration')]

# Monthly Summary

In [136]:
DF

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,2021-10-01 03:00:00.000,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,56.8520,212.247467,14.213000,3.733333,0.25,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,4.0000,14.933333,1.000000,3.733333,0.25,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
2,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,54.0000,175.500000,13.500000,3.250000,0.25,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
3,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,6.0000,19.500000,1.500000,3.250000,0.25,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
4,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,50.0000,282.500000,15.000000,5.650000,0.30,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral,4.936508,5.461814,4.411202,Above,0.188186,0.038121,5.076000,0.113081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16934,1000000005028,2022-08-31 21:04:23.030,2022-08-31 21:59:18.260,2022-09-01 00:04:23.030,8-2022,2022,Q3,8,Aug,31,31,Wed,21,9pm-pt,0am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,54.9205,863.441794,200.459825,15.721667,3.65,4.307306,-0.435043,-0.026926,16.156710,4.110143,17.929893,14.383527,0.000000,0.000000,Neutral,15.754947,16.849181,14.660712,Neutral,0.000000,0.000000,15.703523,0.001155
16935,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,2022-09-01 00:59:18.260,8-2022,2022,Q3,8,Aug,31,31,Wed,21,9pm-pt,0am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,1.0000,15.721667,3.650000,15.721667,3.65,4.307306,-0.435043,-0.026926,16.156710,4.110143,17.929893,14.383527,0.000000,0.000000,Neutral,15.754947,16.849181,14.660712,Neutral,0.000000,0.000000,15.703523,0.001155
16936,1000000005029,2022-08-31 21:59:18.260,2022-08-31 22:55:51.240,2022-09-01 00:59:18.260,8-2022,2022,Q3,8,Aug,31,31,Wed,22,10pm-pt,1am-et,Weekday,Wk,Late Night,Series,Tattoo Fixers on Holiday - Clean,55.0000,508.328333,132.000000,9.242333,2.40,3.850972,-3.943391,-0.299065,13.185725,3.699559,14.781775,11.589674,-2.347341,-0.178021,Below,13.029534,14.078917,11.980151,Below,-2.737818,-0.210124,15.703523,-0.411448
16937,1000000005030,2022-08-31 22:55:51.240,2022-08-31 23:59:59.270,2022-09-01 01

In [137]:
#Add %Minutes Diff CI (3H)

MS= DF.loc[:,('TS','Title','Series/Film','%Minutes Diff CI (3H)')]
MS['Date']=MS['TS'].dt.strftime('%m-%y')
MS= MS.groupby(by=['Date','Title','Series/Film'])[['%Minutes Diff CI (3H)']].mean().reset_index()
MS= MS.pivot(index=['Series/Film','Title'], columns='Date', values=['%Minutes Diff CI (3H)']).reset_index()
MS=MS.droplevel(level=0,axis=1)
MS.columns.values[0]='Series/Film'
MS.columns.values[1]='Title'
MS['Metric']='%Minutes Diff CI (3H)'


In [138]:
# #Add Counts

# MSc= DF.loc[:,('TS','Title','Series/Film','Content ID')]
# MSc['Date']=MSc['TS'].dt.strftime('%m-%y')
# MSc=MSc.pivot_table(index= ['Series/Film','Title'], values='Content ID', columns='Date', aggfunc=pd.Series.count).reset_index()
# MSc['Metric']='Count'


In [139]:
#Add Minutes Aired

MSc= DF.loc[:,('TS','Title','Series/Film','Length(min) in the Hour')]
MSc['Date']=MSc['TS'].dt.strftime('%m-%y')
MSc=MSc.groupby(by=['Date','Title','Series/Film'])[['Length(min) in the Hour']].agg(sum).reset_index()
MSc=MSc.pivot_table(index= ['Series/Film','Title'], values='Length(min) in the Hour', columns='Date').reset_index()
MSc['Metric']='Minutes Aired'

In [140]:
#Add Minutes per Minute

MM= DF.loc[:,('TS','Title','Series/Film','Minutes per Minute')]
MM['Date']=MM['TS'].dt.strftime('%m-%y')
MM= MM.groupby(by=['Date','Title','Series/Film'])[['Minutes per Minute']].mean().reset_index()
MM=MM.pivot_table(index= ['Series/Film','Title'], values='Minutes per Minute', columns='Date').reset_index()
MM['Metric']='Min/Min'


In [141]:
#Add Daypart avg benchmark

MSd= DF.loc[:,('TS','Title','Series/Film','%Minutes Diff (Month DP)')]
MSd['Date']=MSd['TS'].dt.strftime('%m-%y')
MSd= MSd.groupby(by=['Date','Title','Series/Film'])[['%Minutes Diff (Month DP)']].mean().reset_index()
MSd=MSd.pivot_table(index= ['Series/Film','Title'], values='%Minutes Diff (Month DP)', columns='Date').reset_index()
MSd['Metric']='%Minutes Diff (Month DP)'


In [142]:
#Concat Tables

MS_MSc=pd.concat([MS,MSc,MM])
MS_MSc['Average']=MS_MSc.mean(axis=1)


In [143]:
MS_MSc= MS_MSc.reset_index()

In [144]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average')]

elif (Channel == 'ElRey' or Channel == 'SoReal'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average')]
    
else:
    #FilmDetective & Towie
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','08-22',
                         'Metric','Average')]
    

In [145]:
MS_MSc['Count']=MS_MSc.loc[:,MS_MSc.columns[2]:MS_MSc.columns[-3]].count(axis=1)
X_mean=[]

for i in range(0,MS_MSc['Count'].count()):
    X = statistics.mean(list(range(1,MS_MSc['Count'].iloc[i]+1)))
    X_mean = X_mean + [X]

MS_MSc['X_mean']= X_mean

In [146]:
Slope = []
B1e = []
B2e = []
S =[]

for i in range(0,MS_MSc['Count'].count()):
    x = list(range(1, MS_MSc['Count'].iloc[i]+1))
    y = [x for x in list(MS_MSc.loc[i,MS_MSc.columns[2]:MS_MSc.columns[-5]]) if math.isnan(x) == False]
    for u,v in zip(x,y):
        B1 = ((u - MS_MSc['X_mean'].iloc[i]) * (v - MS_MSc['Average'].iloc[i]))
        B2 = ((u - MS_MSc['X_mean'].iloc[i])**2)
        
        B1e = B1e + [B1]
        B2e = B2e + [B2]
        
    S = S + [(sum(B1e)/sum(B2e))]
    B1e.clear()
    B2e.clear()
    
    
MS_MSc['Slope'] = S


<ipython-input-146-eb4776767459>:16: RuntimeWarning: invalid value encountered in double_scalars
  S = S + [(sum(B1e)/sum(B2e))]


In [147]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average','Slope')]

elif (Channel == 'ElRey' or Channel == 'SoReal'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average','Slope')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                         'Metric','Average','Slope')]
    
else:
    MS_MSc=MS_MSc.loc[:,('Series/Film','Title','08-22',
                         'Metric','Average','Slope')]
    
    

# Monthly Summary by Daypart

In [148]:
#Add %Minutes Diff CI

MS1= DF.loc[:,('TS','Daypart','Title','Series/Film','%Minutes Diff CI (3H)')]
MS1['Date']=MS1['TS'].dt.strftime('%m-%y')
MS1= MS1.groupby(by=['Date','Title','Daypart','Series/Film'])[['%Minutes Diff CI (3H)']].mean().reset_index()
MS1= MS1.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values=['%Minutes Diff CI (3H)']).reset_index()
MS1=MS1.droplevel(level=0,axis=1)
MS1.columns.values[0]='Series/Film'
MS1.columns.values[1]='Title'
MS1.columns.values[2]='Daypart'
MS1['Metric']='%Minutes Diff CI (3H)'


In [149]:
# #Add Counts

# MSc1= DF.loc[:,('TS','Daypart','Title','Series/Film','Content ID')]
# MSc1['Date']=MSc1['TS'].dt.strftime('%m-%y')
# MSc1=MSc1.pivot_table(index= ['Series/Film','Title','Daypart'], values='Content ID', columns='Date', aggfunc=pd.Series.count).reset_index()
# MSc1['Metric']='Count'


In [150]:
#Add Minutes Aired

MSc1= DF.loc[:,('TS','Daypart','Title','Series/Film','Length(min) in the Hour')]
MSc1['Date']=MSc1['TS'].dt.strftime('%m-%y')
MSc1=MSc1.groupby(by=['Date','Title','Daypart','Series/Film'])[['Length(min) in the Hour']].agg(sum).reset_index()
MSc1=MSc1.pivot_table(index= ['Series/Film','Title','Daypart'], values='Length(min) in the Hour', columns='Date').reset_index()
MSc1['Metric']='Minutes Aired'

In [151]:
#Add Minutes per Minute

MM1= DF.loc[:,('TS','Daypart','Title','Series/Film','Minutes per Minute')]
MM1['Date']=MM1['TS'].dt.strftime('%m-%y')
MM1= MM1.groupby(by=['Date','Title','Daypart','Series/Film'])[['Minutes per Minute']].mean().reset_index()
MM1= MM1.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values='Minutes per Minute').reset_index()
MM1['Metric']='Min/Min'


In [152]:
#Add %Minutes Diff (Month DP)

MS1dp= DF.loc[:,('TS','Daypart','Title','Series/Film','%Minutes Diff (Month DP)')]
MS1dp['Date']=MS1dp['TS'].dt.strftime('%m-%y')
MS1dp= MS1dp.groupby(by=['Date','Title','Daypart','Series/Film'])[['%Minutes Diff (Month DP)']].mean().reset_index()
MS1dp= MS1dp.pivot(index=['Series/Film','Title','Daypart'], columns='Date', values='%Minutes Diff (Month DP)').reset_index()
MS1dp['Metric']='%Minutes Diff (Month DP)'

In [153]:
#Concat Tables

MS1_MSc1=pd.concat([MS1,MSc1,MM1])
MS1_MSc1['Average']=MS1_MSc1.mean(axis=1)
MS1_MSc1.reset_index(drop=True, inplace=True)

In [154]:
MS1_MSc1= MS1_MSc1.reset_index()

In [155]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average')]

elif (Channel == 'ElRey' or Channel == 'SoReal'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average')]
    
else:
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','08-22',
                             'Metric','Average')]

In [156]:
MS1_MSc1['Count']=MS1_MSc1.loc[:,MS1_MSc1.columns[3]:MS1_MSc1.columns[-3]].count(axis=1)
X_mean=[]

for i in range(0,MS1_MSc1['Count'].count()):
    X = statistics.mean(list(range(1,MS1_MSc1['Count'].iloc[i]+1)))
    X_mean = X_mean + [X]

MS1_MSc1['X_mean']= X_mean

In [157]:
Slope = []
B1e = []
B2e = []
S =[]

for i in range(0,MS1_MSc1['Count'].count()):
    x = list(range(1, MS1_MSc1['Count'].iloc[i]+1))
    y = [x for x in list(MS1_MSc1.loc[i,MS1_MSc1.columns[3]:MS1_MSc1.columns[-5]]) if math.isnan(x) == False]
    for u,v in zip(x,y):
        B1 = ((u - MS1_MSc1['X_mean'].iloc[i]) * (v - MS1_MSc1['Average'].iloc[i]))
        B2 = ((u - MS1_MSc1['X_mean'].iloc[i])**2)
        
        B1e = B1e + [B1]
        B2e = B2e + [B2]
        
    S = S + [(sum(B1e)/sum(B2e))]
    B1e.clear()
    B2e.clear()
    
    
MS1_MSc1['Slope'] = S


<ipython-input-157-344369c2615a>:16: RuntimeWarning: invalid value encountered in double_scalars
  S = S + [(sum(B1e)/sum(B2e))]


In [158]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','07-21','08-21','09-21','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average','Slope')]

elif (Channel == 'ElRey' or Channel == 'SoReal'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','10-21','11-21','12-21','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average','Slope')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','01-22','02-22','03-22','04-22','05-22','06-22','07-22','08-22',
                             'Metric','Average','Slope')]
    
else:
    MS1_MSc1=MS1_MSc1.loc[:,('Series/Film','Title','Daypart','08-22',
                             'Metric','Average','Slope')]


# Platform Monthly Total Minutes

In [159]:
Platform['Year']=Platform['Date'].dt.year

<ipython-input-159-21b592892849>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Platform['Year']=Platform['Date'].dt.year


In [160]:
CondMonth=[
    Platform['Month'] == 1,Platform['Month'] == 2,Platform['Month'] == 3,
    Platform['Month'] == 4,Platform['Month'] == 5,Platform['Month'] == 6,
    Platform['Month'] == 7,Platform['Month'] == 8,Platform['Month'] == 9,
    Platform['Month'] == 10,Platform['Month'] == 11,Platform['Month'] == 12,
]

ValueM=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

Platform['Months']= np.select(CondMonth,ValueM)

<ipython-input-160-c9b892d431c8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Platform['Months']= np.select(CondMonth,ValueM)


In [161]:
Platform= Platform.pivot_table(index='Platform',values='Total Viewership Minutes',columns=['Year','Months'], aggfunc=pd.Series.sum).reset_index()

In [162]:
if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    Platform.columns=('Platform','Aug-21','Dec-21','Jul-21','Nov-21','Oct-21','Sep-21','Apr-22','Aug-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22')
    Platform=Platform.loc[:,('Platform','Jul-21','Aug-21','Sep-21','Oct-21','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22')]

elif (Channel == 'ElRey' or Channel == 'SoReal'):
    Platform.columns=('Platform','Dec-21','Nov-21','Oct-21','Apr-22','Aug-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22')
    Platform=Platform.loc[:,('Platform','Oct-21','Nov-21','Dec-21','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22')]
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    Platform.columns=('Platform','Apr-22','Aug-22','Feb-22','Jan-22','Jul-22','Jun-22','Mar-22','May-22')
    Platform=Platform.loc[:,('Platform','Jan-22','Feb-22','Mar-22','Apr-22','May-22','Jun-22','Jul-22','Aug-22')]
    
else:
    Platform.columns=('Platform','Aug-22')
    Platform=Platform.loc[:,('Platform','Aug-22')]
    

# Total Minutes Per Month for each Content

In [163]:
TotChnMonMin=DF.groupby(['Year','Month'])['Total Minutes in Hour'].agg(sum).reset_index()

In [164]:
TotMinC= DF.groupby(by=['Year','Month','Series/Film','Title'])['Total Minutes in Hour'].agg(sum).reset_index()

In [165]:
TotMinC=pd.merge(TotMinC, TotChnMonMin, on=['Year','Month'], how='left')

In [166]:
TotMinC.rename(columns={'Total Minutes in Hour_x':'Tot Content Month Min','Total Minutes in Hour_y':'Tot Chn Month Min'}, inplace=True)

In [167]:
# Percent Content minutes from total monthly Minutes

TotMinC['% of Mon Min']= TotMinC['Tot Content Month Min']/TotMinC['Tot Chn Month Min']

In [168]:
TotMinCP= TotMinC.pivot(index=['Series/Film','Title'], columns=['Year','Month'], values='% of Mon Min').reset_index()

In [169]:
# _____________________________________________RENAME COLUMNS CAREFUl____________________________________


if (Channel == 'CONtv' or  Channel == 'ComedyDynamics' or Channel == 'Docurama' or Channel == 'Dove'):   
    TotMinCP.columns=['Series/Film','Title','7-2021','8-2021','9-2021','10-2021','11-2021','12-2021','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022']
    TotMinCP['Average']= TotMinCP.loc[:,'7-2021':'8-2022'].mean(axis=1)
    
elif (Channel == 'ElRey' or Channel == 'SoReal'):
    TotMinCP.columns=['Series/Film','Title','10-2021','11-2021','12-2021','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022']
    TotMinCP['Average']= TotMinCP.loc[:,'10-2021':'8-2022'].mean(axis=1)
    
elif (Channel == 'BloodyDisgusting' or Channel == 'Towie'):
    TotMinCP.columns=['Series/Film','Title','1-2022','2-2022','3-2022','4-2022','5-2022','6-2022','7-2022','8-2022']
    TotMinCP['Average']= TotMinCP.loc[:,'1-2022':'8-2022'].mean(axis=1)
    
else:
    TotMinCP.columns=['Series/Film','Title','8-2022']
    #TotMinCP['Average']= TotMinCP.loc[:,'8-2022'].mean(axis=1)
    


In [170]:
if Timezone == 'East':
    DF.rename(columns={'Hour(24-Pac)':'Hour(24-East)'},inplace=True)

# Last Look

In [171]:
DF.head(10)

,Content ID,TS,TS End,TS_East,M_Y,Year,Quarter,Month,Month(name),Day,Day (adj.),Weekday(name),Hour(24-Pac),Pacific_Time,East_Time,Weekdays/Weekends,Wk/Wknd,Daypart,Series/Film,Title,Length(min) in the Hour,Total Minutes in Hour,Total Sessions in Hour,Minutes per Minute,Sessions per Minute,Session Duration,Minutes Diff,%Minutes Diff,Hourly AVG,Hourly STD,Upper CI(95%),Lower CI(95%),Minutes Diff CI,%Minutes Diff CI,Above/Below/Neutral,Mean (H3),Upper CI(95%) (3H),Lower CI(95%) (3H),Above/Below/Neutral (3H),Minutes Diff CI (3H),%Minutes Diff CI (3H),Mean (Month DP),%Minutes Diff (Month DP)
0,CEG10029784,2021-10-01 00:00:00.000,2021-10-01 00:56:51.120,2021-10-01 03:00:00.000,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,56.852,212.247467,14.213000,3.733333,0.250000,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
1,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,0,0am-pt,3am-et,Weekday,Wk,Overnight,Series,World of Weird,4.000,14.933333,1.000000,3.733333,0.250000,14.933333,-1.421429,-0.275751,5.154762,2.012303,6.068133,4.241391,-0.508058,-0.098561,Below,5.230423,5.707047,4.753800,Below,-1.020467,-0.195102,5.076000,-0.264513
2,CEG10029785,2021-10-01 00:56:51.120,2021-10-01 01:54:00.070,2021-10-01 03:56:51.120,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,54.000,175.500000,13.500000,3.250000,0.250000,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
3,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,1,1am-pt,4am-et,Weekday,Wk,Overnight,Series,World of Weird,6.000,19.500000,1.500000,3.250000,0.250000,13.000000,-1.875397,-0.365903,5.125397,2.115187,6.085466,4.165328,-0.915328,-0.178587,Below,5.092328,5.599321,4.585335,Below,-1.335335,-0.262225,5.076000,-0.359732
4,CEG10029786,2021-10-01 01:54:00.070,2021-10-01 02:50:56.240,2021-10-01 04:54:00.070,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,50.000,282.500000,15.000000,5.650000,0.300000,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral,4.936508,5.461814,4.411202,Above,0.188186,0.038121,5.076000,0.113081
5,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,2021-10-01 05:50:56.240,10-2021,2021,Q4,10,Oct,1,5,Fri,2,2am-pt,5am-et,Weekday,Wk,Overnight,Series,World of Weird,10.000,56.500000,3.000000,5.650000,0.300000,18.833333,0.653175,0.130718,4.996825,2.128221,5.962810,4.030841,0.000000,0.000000,Neutral,4.936508,5.461814,4.411202,Above,0.188186,0.038121,5.076000,0.113081
6,CEG10029787,2021-10-01 02:50:56.240,2021-10-01 03:46:39.050,2021-10-01 05:50:56.240,10-2021,2021,Q4,10,Oct,1,5,Fri,3,3am-pt,6am-et,Weekday,Wk,Weekday Morning,Series,World of Weird,46.000,288.266667,11.500000,6.266667,0.250000,25.066667,1.579365,0.336945,4.687302,2.217837,5.693963,3.680640,0.572704,0.122182,Above,4.638624,5.135382,4.141867,Above,1.131285,0.243884,4.432895,0.413674
7,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,2021-10-01 06:46:39.050,10-2021,2021,Q4,10,Oct,1,5,Fri,3,3am-pt,6am-et,Weekday,Wk,Weekday Morning,Series,17 Kids and Counting,14.000,87.733333,3.500000,6.266667,0.250000,25.066667,1.579365,0.336945,4.687302,2.217837,5.693963,3.680640,0.572704,0.122182,Above,4.638624,5.135382,4.141867,Above,1.131285,0.243884,4.432895,0.413674
8,CEG10028439,2021-10-01 03:46:39.050,2021-10-01 04:44:47.280,2021-10-01 06:46:39.050,10-2021,2021,Q4,10,Oct,1,5,Fri,4,4am-pt,7am-et,Weekday,Wk,Weekday Morning,Series,17 Kids and Counting,44.000,249.333333,7.333333,5.666667,0.166667,34.000000,1.434921,0.339085,4.231746,1.669156,4.989364,3.474128,0.677302,0.160053,Above,4.464550,4.91

In [172]:
CSsumm.head(10)

,Series/Film,Title,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
42,Series,Tattoo Fixers - Clean,1995.0,1047.0,418.0,530.0,18.416657,0.116176,0.175738,47.478403,60656.952500,1.112132e+06,196972.767744,5.646119
26,Series,Hoarders Get Your House In Order,911.0,526.0,179.0,206.0,20.153264,0.141927,0.205703,36.492856,26651.207833,5.413802e+05,121224.132022,4.465944
27,Series,Hoarders SOS,1163.0,542.0,275.0,346.0,19.616242,0.079117,0.134280,11.840254,35041.961333,6.893791e+05,142650.126981,4.832657
38,Series,Perfect Vagina,333.0,167.0,80.0,86.0,11.751974,0.125007,0.159681,7.693014,10386.000000,1.262600e+05,22006.566667,5.737379
30,Series,"Kinky Sex Survey, The",292.0,139.0,81.0,72.0,11.757354,0.110840,0.134191,5.366258,9751.012000,1.158808e+05,22040.084211,5.257732
20,Series,Danger Beach,302.0,148.0,73.0,81.0,7.543424,0.061683,0.085913,3.133804,5648.351667,4.259969e+04,4107.894361,10.370201
12,Series,Air Rescue,775.0,324.0,200.0,251.0,9.322344,0.046717,0.048005,2.530257,14406.285833,1.345945e+05,19315.639242,6.968161
45,Series,The Cops - Clean,136.0,67.0,43.0,26.0,14.006143,0.081926,0.074172,2.296929,3807.770333,5.537587e+04,18683.417917,2.963905
11,Series,Addicted to Boob Jobs,342.0,145.0,85.0,112.0,10.910143,0.077203,0.083245,1.914487,10263.000000,1.120816e+05,19632.800000,5.708895
32,Series,"Love Me, Love My Doll",259.0,104.0,65.0,90.0,9.682793,0.050567,0.053409,0.530270,9704.207167,9.214276e+04,12088.315750,7.622464


In [173]:
CSDsumm.head(15)

,Series/Film,Title,Daypart,Count,Above,Neutral,Below,Minutes per Minute,%Minutes Diff CI (3H),%Minutes Diff (Month DP),Combined Score,Total Min Aired,Total Min Viewed,Total Sessions,Session Duration
0,Film,Kids With Tourette's - In Their Own Words,Late Night,1.0,0.0,0.0,1.0,7.480000,-0.337777,-0.542782,-0.337777,3.000000,22.440000,5.150000,4.357282
1,Film,Kids With Tourette's - In Their Own Words,Overnight,1.0,0.0,0.0,1.0,1.427833,-0.751959,-0.815219,-0.751959,60.000000,85.670000,74.000000,1.157703
2,Film,Kids With Tourette's - In Their Own Words,Overnight (wknd),1.0,0.0,0.0,1.0,4.457167,-0.267197,-0.539006,-0.267197,11.000000,49.028833,9.900000,4.952407
3,Film,Kids With Tourette's - In Their Own Words,Weekday Day,23.0,8.0,2.0,13.0,9.810507,-0.002934,0.011580,-0.013395,686.168000,6306.251894,937.804267,6.724486
4,Film,Kids With Tourette's - In Their Own Words,Weekday Fringe,9.0,1.0,2.0,6.0,13.269944,-0.081987,-0.163477,-0.318838,270.000000,3781.964167,691.183333,5.471724
5,Film,Kids With Tourette's - In Their Own Words,Weekend Morning,1.0,0.0,0.0,1.0,4.132333,-0.322214,-0.548285,-0.322214,44.000000,181.822667,51.333333,3.542000
6,Film,The Boy Who Can't Forget,Weekday Day,27.0,7.0,5.0,15.0,8.030340,-0.103028,-0.113966,-0.671590,807.813500,6411.860454,934.772992,6.859270
7,Film,The Boy Who Can't Forget,Weekday Fringe,4.0,1.0,2.0,1.0,14.714667,-0.058456,-0.070006,0.000000,166.000000,2019.318667,346.600000,5.826078
8,Film,The Boy Who Can't Forget,Weekend Day,1.0,0.0,1.0,0.0,14.477667,0.000000,-0.239220,0.000000,40.000000,579.106667,97.333333,5.949726
9,Film,The Boy Who Can't Forget,Weekend Morning,1.0,0.0,1.0,0.0,10.186667,0.000000,0.113527,0.000000,17.000000,173.173333,41.650000,4.157823


In [174]:
MS_MSc.head(15)

Date,Series/Film,Title,10-21,11-21,12-21,01-22,02-22,03-22,04-22,05-22,06-22,07-22,08-22,Metric,Average,Slope
0,Film,Kids With Tourette's - In Their Own Words,NaN,NaN,-0.168063,0.097854,-0.089640,0.024678,-0.294705,0.052603,NaN,-0.122988,0.090739,%Minutes Diff CI (3H),-0.051190,0.009699
1,Film,The Boy Who Can't Forget,NaN,0.374683,-0.083936,NaN,-0.111329,0.011566,-0.187844,-0.173725,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.028431,-0.083739
2,Series,16 Kids and Counting,0.023527,0.126332,0.074113,-0.000920,-0.028707,-0.158629,-0.070085,NaN,-0.482705,-0.236284,0.115923,%Minutes Diff CI (3H),-0.063743,-0.029262
3,Series,17 Kids and Counting,-0.126881,0.150769,-0.009492,0.019009,0.028328,-0.087229,0.041054,-0.056046,-0.064970,-0.007344,-0.006088,%Minutes Diff CI (3H),-0.010808,-0.003021
4,Series,18 Kids and Counting,0.264002,0.172674,0.049162,0.075522,0.038045,-0.012803,0.048830,-0.028382,-0.097251,0.017787,-0.040995,%Minutes Diff CI (3H),0.044236,-0.025280
5,Series,19 Kids and Counting,0.440803,0.165202,0.065793,0.086180,0.096297,-0.056333,0.032151,0.014082,-0.094532,-0.084133,-0.088568,%Minutes Diff CI (3H),0.052449,-0.039396
6,Series,20 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,-0.028675,0.028694,-0.064006,-0.028229,-0.101823,%Minutes Diff CI (3H),-0.038808,-0.020322
7,Series,21 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.015037,-0.050127,-0.001768,0.028707,%Minutes Diff CI (3H),-0.009556,0.017959
8,Series,22 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.042407,-0.038256,-0.045720,%Minutes Diff CI (3H),-0.042128,-0.001657
9,Series,A Model Life,-0.268525,NaN,-0.521075,NaN,NaN,NaN,-0.065865,NaN,-0.162101,NaN,-0.503319,%Minutes Diff CI (3H),-0.304177,-0.011061


In [175]:
MS1_MSc1

Date,Series/Film,Title,Daypart,10-21,11-21,12-21,01-22,02-22,03-22,04-22,05-22,06-22,07-22,08-22,Metric,Average,Slope
0,Film,Kids With Tourette's - In Their Own Words,Late Night,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.337777,NaN,%Minutes Diff CI (3H),-0.337777,NaN
1,Film,Kids With Tourette's - In Their Own Words,Overnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.751959,NaN,%Minutes Diff CI (3H),-0.751959,NaN
2,Film,Kids With Tourette's - In Their Own Words,Overnight (wknd),NaN,NaN,NaN,NaN,NaN,NaN,-0.267197,NaN,NaN,NaN,NaN,%Minutes Diff CI (3H),-0.267197,NaN
3,Film,Kids With Tourette's - In Their Own Words,Weekday Day,NaN,NaN,-0.161108,0.097854,-0.089640,NaN,NaN,0.052603,NaN,0.087951,0.304089,%Minutes Diff CI (3H),0.048625,0.069672
4,Film,Kids With Tourette's - In Their Own Words,Weekday Fringe,NaN,NaN,-0.178496,NaN,NaN,0.024678,NaN,NaN,NaN,NaN,-0.122612,%Minutes Diff CI (3H),-0.092143,0.027942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,Series,World of Weird,Weekend Day,NaN,9.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Min/Min,9.150000,NaN
1271,Series,World of Weird,Weekend Morning,7.466667,6.505556,7.942083,NaN,6.368889,13.056833,12.249000,4.320667,NaN,NaN,NaN,Min/Min,8.272813,0.255844
1272,Series,"Young, Free and Single: Live",Late Night,3.933333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Min/Min,3.933333,NaN
1273,Series,"Young, Free and Single: Live",Prime,2.763542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Min/Min,2.763542,NaN


In [176]:
Platform

,Platform,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,May-22,Jun-22,Jul-22,Aug-22
0,Klowd,10039.0,23352.0,18405.93,20934.63,20887.79,17330.96,16017.47,NaN,NaN,NaN,NaN
1,LG,645.0,NaN,82828.54,137423.54,148431.86,561953.91,361451.56,331815.95,623639.76,312470.81,257032.95
2,Littlstar,45334.0,43224.0,44470.30,9438.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Plex,145452.0,242959.0,266378.37,249593.22,240525.96,237206.45,272381.10,239324.42,229322.95,277198.22,285213.64
4,Stremium,37880.0,14177.0,11940.55,18466.04,12992.37,16365.65,21370.75,19011.87,23375.31,29514.51,16015.54
5,TCL,2734.0,2805.0,2226.91,597.24,4434.70,5163.52,2889.94,18000.02,14357.12,9564.18,14376.29


In [177]:
TotMinCP.head(10)

,Series/Film,Title,10-2021,11-2021,12-2021,1-2022,2-2022,3-2022,4-2022,5-2022,6-2022,7-2022,8-2022,Average
0,Film,Kids With Tourette's - In Their Own Words,NaN,NaN,0.004203,0.002579,0.001592,0.003185,0.000333,0.002740,NaN,0.002406,0.001318,0.002294
1,Film,The Boy Who Can't Forget,NaN,0.003007,0.005120,NaN,0.002984,0.001566,0.002731,0.002535,NaN,NaN,NaN,0.002991
2,Series,16 Kids and Counting,0.028227,0.025318,0.046330,0.055719,0.047982,0.018887,0.035088,NaN,0.000420,0.004148,0.007842,0.026996
3,Series,17 Kids and Counting,0.005417,0.010021,0.010050,0.008585,0.011045,0.011622,0.009498,0.007351,0.004726,0.009036,0.007075,0.008584
4,Series,18 Kids and Counting,0.006328,0.010267,0.011164,0.008310,0.012076,0.011325,0.008549,0.009854,0.005550,0.006218,0.007701,0.008849
5,Series,19 Kids and Counting,0.007028,0.010150,0.010900,0.008354,0.012907,0.012042,0.007535,0.015576,0.007827,0.006960,0.008537,0.009802
6,Series,20 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,0.003458,0.017703,0.009656,0.009288,0.008829,0.009787
7,Series,21 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.016613,0.009302,0.007231,0.011920,0.011267
8,Series,22 Kids and Counting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031906,0.021950,0.054933,0.036263
9,Series,A Model Life,0.008453,NaN,0.004700,NaN,NaN,NaN,0.015007,NaN,0.002897,NaN,0.002068,0.006625


In [178]:
output= pd.ExcelWriter(Channel+ ' Content Full Summary out ' + Date_span + '.xlsx', engine='xlsxwriter')

DF.to_excel(output, sheet_name= 'DF',index=False)
CSsumm.to_excel(output, sheet_name= 'Cont Summ',index=False)
CSDsumm.to_excel(output, sheet_name='Cont Summ DayP', index=False)
MS_MSc.to_excel(output, sheet_name='Monthly Trend', index=False)
MS1_MSc1.to_excel(output, sheet_name='Monthly DayP Trend',index=False)
Platform.to_excel(output, sheet_name='Graphs',index=False)
TotMinCP.to_excel(output, sheet_name='% Month Min',index=False)
#Bucks.to_excel(output, sheet_name='Bucket',index=False)



output.save()